# <center><b><font color = 'green'>Modelos Preditivos para Usinas Geradoras de Energia Elétrica</font><b></center>

##### Teste com todos os modelos e todas as usinas
##### Data: 08/2023

## SUMÁRIO
* [1. Configurações Gerais](#chapter_1)
* [2. Visualização do Histórico por Usina](#chapter_2)
    * **[2.1. Usina 1](#chapter_2_1)**
         * [2.1.1. LSTM](#chapter_2_1_1)
         * [2.1.2. Holt-Winters](#chapter_2_1_2)
         * [2.1.3. SARIMAX](#chapter_2_1_3)
         * [2.1.4. TensorFlow](#chapter_2_1_4)
         * [2.1.5. XGBoost](#chapter_2_1_5)
         * [2.1.6. MLP](#chapter_2_1_6)
         * [2.1.7. Random Forest](#chapter_2_1_7)
         * [2.1.8. Regressão Linear](#chapter_2_1_8)
    * **[2.2. Usina 2](#chapter_2_2)**
         * [2.2.1. LSTM](#chapter_2_2_1)
         * [2.2.2. Holt-Winters](#chapter_2_2_2)
         * [2.2.3. SARIMAX](#chapter_2_2_3)
         * [2.2.4. TensorFlow](#chapter_2_2_4)
         * [2.2.5. XGBoost](#chapter_2_2_5)
         * [2.2.6. MLP](#chapter_2_2_6)
         * [2.2.7. Random Forest](#chapter_2_2_7)
         * [2.2.8. Regressão Linear](#chapter_2_2_8) 
    * **[2.3. Usina 3](#chapter_2_3)**
         * [2.3.1. LSTM](#chapter_2_3_1)
         * [2.3.2. Holt-Winters](#chapter_2_3_2)
         * [2.3.3. SARIMAX](#chapter_2_3_3)
         * [2.3.4. TensorFlow](#chapter_2_3_4)
         * [2.3.5. XGBoost](#chapter_2_3_5)
         * [2.3.6. MLP](#chapter_2_3_6)
         * [2.3.7. Random Forest](#chapter_2_3_7)
         * [2.3.8. Regressão Linear](#chapter_2_3_8) 
    * **[2.4. Usina 4](#chapter_2_4)**
         * [2.4.1. LSTM](#chapter_2_4_1)
         * [2.4.2. Holt-Winters](#chapter_2_4_2)
         * [2.4.3. SARIMAX](#chapter_2_4_3)
         * [2.4.4. TensorFlow](#chapter_2_4_4)
         * [2.4.5. XGBoost](#chapter_2_4_5)
         * [2.4.6. MLP](#chapter_2_4_6)
         * [2.4.7. Random Forest](#chapter_2_4_7)
         * [2.4.8. Regressão Linear](#chapter_2_4_8) 
* [3. Resultados](#chapter_3)
    * [3.1. Usina 1](#chapter_3_1)
    * [3.2. Usina 2](#chapter_3_2)
    * [3.3. Usina 3](#chapter_3_3)
    * [3.4. Usina 4](#chapter_3_4)

### <b>1. Configurações Gerais</b> <a class = "anchor" id = "chapter_1"></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.offline as py
import plotly.graph_objs as go
import warnings

import xgboost as xgb
import lightgbm as lgb
import tensorflow_probability as tfp

warnings.filterwarnings("ignore")

In [ ]:
from plotly.subplots import make_subplots

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import ParameterGrid
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

from sklearn import metrics
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

from pmdarima import auto_arima
from statsmodels.tsa.holtwinters import ExponentialSmoothing

In [ ]:
# Para salvar os modelos

import pickle

In [ ]:
# Lendo os dados do arquivo em formato csv 

df = pd.read_csv('usinas_selec.csv')
df.head()

In [ ]:
# Convertendo a columa 'Competencia' para o formato datetime
df['Competência'] = pd.to_datetime(df['Competência'], format='%d/%m/%Y')
 
# Verificando o formato da coluna "Competencia"
df.info()

In [ ]:
# Excluindo coluna com indices extra

df = df.drop(df.columns[0], axis=1)

In [ ]:
# Colocando a 'Competencia' como índice

df= df.set_index('Competência')
df.head()

### <b>2. Visualização do Histórico por Usina</b> <a class = "anchor" id = "chapter_2"></a>

### <b>2.1. Usina 1</b> <a class = "anchor" id = "chapter_2_1"></a>

In [ ]:
usina1 = df[df['Usina'] == '1'].groupby('Competência').agg({'Compensada':'sum', 'Acumulado':'sum', 
                                                             'Energia_Ativa':'sum'})

usina1

In [ ]:
# Descricao estatistica dos dados

usina1.describe()

In [ ]:
fig = go.Figure()

# Configurando o gráfico de barras
fig.add_trace(go.Bar(x = usina1.index,
                     y = usina1['Compensada'],
                     text = usina1['Compensada'],
                     name = 'Gráfico de barras',
                     textposition = 'inside',
                     textfont_size = 12,
                     marker_color = '#266B6E'))

# Configurando o gráfico de linhas
fig.add_trace(go.Scatter(x = usina1.index,
                         y = usina1['Compensada'],
                         name = 'Gráfico de linhas',
                         mode = 'lines + markers',
                         marker_color = 'red'))


# Configurando o título do gráfico
fig.update_layout(title_text = f'Energia Compensada - 1',
                  title_font_size = 30,
                  title_x = 0.5)

# Configurando o título dos eixos
fig.update_layout(yaxis_title = 'Quantidade de energia',
                  yaxis_title_font_size = 18,
                  xaxis_title = 'Meses',
                  showlegend = False)

# Exibindo as grades de fundo
fig.update_xaxes(showgrid = True)

# Plotando o gráfico
fig.show()

### <b>2.1.1. LSTM</b> <a class = "anchor" id = "chapter_2_1_1"></a>

In [ ]:
# Copia do dataset principal

usina1_lstm = usina1.copy(deep = True)

In [ ]:
# Separando quais dados serao utilizados para rodar o modelo

usina1_lstm = usina1_lstm.loc[:,['Compensada']]
usina1_lstm

In [ ]:
# Definindo a frequencia dos dados

usina1_lstm = usina1_lstm.asfreq('MS')

In [ ]:
# Normalizando os dados

scaler_lstm = MinMaxScaler(feature_range = (0, 1))
data_lstm1 = scaler_lstm.fit_transform(usina1_lstm)

In [ ]:
# Dividindo os dados em conjuntos de treino e teste

train_size_lstm1 = int(len(data_lstm1) * 0.7)               #70% para treino e 30% para teste
training_set_lstm1, test_set_lstm1 = data_lstm1[:train_size_lstm1], data_lstm1[train_size_lstm1:]

In [ ]:
# Verificando as dimensoes dos conjuntos

training_set_lstm1.shape, test_set_lstm1.shape

In [ ]:
# Atribuindo valor a janela deslizante

lookback_window = 1

In [ ]:
# Criando a funcao de janela deslizante

def create_dataset(data, lookback_window):
    X, y = [], []

    for i in range(len(data) - lookback_window):
        X.append(data[i:(i + lookback_window), :])
        y.append(data[i + lookback_window, 0])

    return np.array(X), np.array(y)

In [ ]:
# Ajustando os conjuntos de treino e teste

X_train_lstm1, y_train_lstm1 = create_dataset(training_set_lstm1, lookback_window)
X_test_lstm1, y_test_lstm1 = create_dataset(test_set_lstm1, lookback_window)

In [ ]:
# Verificando as dimensoes dos conjuntos

X_test_lstm1.shape, y_test_lstm1.shape

In [ ]:
# Ajustando parametros do modelo LSTM

best_model_lstm = Sequential()
best_model_lstm.add(LSTM(4, input_shape = (lookback_window, 1)))
best_model_lstm.add(Dense(1))
best_model_lstm.compile(loss = 'mean_squared_error', optimizer = 'adam', run_eagerly=True)

history_lstm1 = best_model_lstm.fit(X_train_lstm1,
                                   y_train_lstm1,
                                   epochs = 100,
                                   batch_size = 1,
                                   validation_split = 0.2,
                                   verbose = 2)

In [ ]:
# Rodando modelo para conjunto de teste

y_pred_lstm1 = best_model_lstm.predict(X_test_lstm1)

y_pred_lstm1 = scaler_lstm.inverse_transform(y_pred_lstm1)
y_test_lstm1 = scaler_lstm.inverse_transform(y_test_lstm1.reshape(-1, 1))

In [ ]:
# Criando dataframe com as predicoes

timestamp1 = pd.DataFrame(usina1_lstm.index[-len(X_test_lstm1):])

y_test_df_lstm1 = pd.concat([timestamp1, pd.DataFrame(y_test_lstm1)], axis = 1).rename(columns = {0: 'Compensada'}).dropna()
y_pred_df_lstm1 = pd.concat([timestamp1, pd.DataFrame(y_pred_lstm1)], axis = 1).rename(columns = {0: 'Compensada_predito'}).dropna()


# Ajuste nas datas para corresponderem às datas reais dos registros de teste

y_test_df_lstm1.index = usina1_lstm.index[-len(X_test_lstm1):]
y_pred_df_lstm1.index = usina1_lstm.index[-len(X_test_lstm1):]

In [ ]:
fig1 = go.Figure()

# Configurando o gráfico de linhas
fig1.add_trace(go.Scatter(x = y_test_df_lstm1['Competência'], y = y_test_df_lstm1['Compensada'], mode = 'lines + markers', name = 'Compensada', marker_color = '#266B6E'))
fig1.add_trace(go.Scatter(x = y_pred_df_lstm1['Competência'], y = y_pred_df_lstm1['Compensada_predito'], mode = 'lines + markers', name = 'Predição LSTM', marker_color = '#FF007D'))

# Configurando o título do gráfico
fig1.update_layout(title_text = f'Energia Compensada - 1',
                  title_font_size = 30,
                  title_x = 0.5)

# Configurando o título dos eixos
fig1.update_layout(xaxis_title_font_size = 18,
                  yaxis_title = 'Quantidade de Energia',
                  yaxis_title_font_size = 18,
                  showlegend = True,
                  legend_title = 'Legenda')

# Exibindo as grades de fundo
fig1.update_xaxes(showgrid = True)

# Plotando o gráfico
fig1.show()

In [ ]:
# Obtendo metricas de avaliacao

erros_lstm1 = pd.DataFrame({'Métricas avaliadas': ['MAE', 'MAPE', 'WMAPE', 'MSE', 'RMSE'],
                                'Valores obtidos': [round(mean_absolute_error(y_test_lstm1, y_pred_lstm1), 2),
                                                    round(mean_absolute_percentage_error(y_test_lstm1, y_pred_lstm1) * 100, 2),
                                                    round(mean_absolute_percentage_error(y_test_lstm1, y_pred_lstm1, sample_weight = y_test_lstm1) * 100, 2),
                                                    round(mean_squared_error(y_test_lstm1, y_pred_lstm1, squared = True), 2),
                                                    round(mean_squared_error(y_test_lstm1, y_pred_lstm1, squared = False), 2)]},
                            index = range(0, 5))


erros_lstm1

### <b>2.1.2. Holt-Winters</b> <a class = "anchor" id = "chapter_2_1_2"></a>

In [ ]:
# Copia do dataset principal

usina1_holtw = usina1.copy(deep = True)

In [ ]:
# Separando quais dados serao utilizados para rodar o modelo

usina1_holtw = usina1_holtw.loc[:,['Compensada']]

In [ ]:
# Definindo a frequência dos dados

usina1_holtw = usina1_holtw.asfreq('MS')

In [ ]:
# Dividindo dados em conjunto de treino (70%)

training_set_holtwinters1 = usina1_holtw.iloc[:-4, :]
training_set_holtwinters1

In [ ]:
# Dividindo dados em conjunto de teste (30%)

test_set_holtwinters1 = usina1_holtw.iloc[-4:, :]
test_set_holtwinters1

In [ ]:
# Aplicando o modelo

model_holtwinters1 = ExponentialSmoothing(endog = training_set_holtwinters1['Compensada'],     
                                         trend = 'add',                             
                                         seasonal = 'add',
                                         seasonal_periods = 4).fit()               

In [ ]:
predictions_holtwinters1 = pd.Series(model_holtwinters1.forecast(steps = len(test_set_holtwinters1))).rename('HW')
predictions_holtwinters1.index = test_set_holtwinters1.index

In [ ]:
fig2 = go.Figure()

# Configurando o gráfico de linhas
fig2.add_trace(go.Scatter(x = test_set_holtwinters1.index, y = test_set_holtwinters1['Compensada'], mode = 'lines + markers', name = 'Compensada', marker_color = '#266B6E'))
fig2.add_trace(go.Scatter(x = predictions_holtwinters1.index, y = predictions_holtwinters1, mode = 'lines + markers', name = 'Predição Holt-Winters', marker_color = '#FF007D'))

# Configurando o título do gráfico
fig2.update_layout(title_text = f'Energia Compensada - 1',
                  title_font_size = 30,
                  title_x = 0.5)

# Configurando o título dos eixos
fig2.update_layout(xaxis_title_font_size = 18,
                  yaxis_title = 'Quantidade de Energia',
                  yaxis_title_font_size = 18,
                  showlegend = True,
                  legend_title = 'Legenda')

# Exibindo as grades de fundo
fig2.update_xaxes(showgrid = True)

# Plotando o gráfico
fig2.show()

In [ ]:
# Obtendo metricas de avaliacao

erros_hw1 = pd.DataFrame({'Métricas avaliadas': ['MAE', 'MAPE', 'WMAPE', 'MSE', 'RMSE'],
                        'Valores obtidos': [round(mean_absolute_error(test_set_holtwinters1['Compensada'], predictions_holtwinters1), 2),
                                            round(mean_absolute_percentage_error(test_set_holtwinters1['Compensada'], predictions_holtwinters1) * 100, 2),
                                            round(mean_absolute_percentage_error(test_set_holtwinters1['Compensada'], predictions_holtwinters1, sample_weight = test_set_holtwinters1['Compensada']) * 100, 2),
                                            round(mean_squared_error(test_set_holtwinters1['Compensada'], predictions_holtwinters1, squared = True), 2),
                                            round(mean_squared_error(test_set_holtwinters1['Compensada'], predictions_holtwinters1, squared = False), 2)]},
                        index = range(0, 5))

erros_hw1

### <b>2.1.3. SARIMAX</b> <a class = "anchor" id = "chapter_2_1_3"></a>

In [ ]:
# Copia do dataset principal

usina1_sarimax = usina1.copy(deep = True)

In [ ]:
# Definindo a frequencia dos dados

usina1_sarimax = usina1_sarimax.asfreq('MS')

In [ ]:
# Definindo o conjunto de treino (70%)

training_set_sarimax1 = usina1_sarimax.iloc[:-4, :]
training_set_sarimax1

In [ ]:
# Definindo o conjunto de teste (30%)

test_set_sarimax1 = usina1_sarimax.iloc[-4:, :]
test_set_sarimax1

In [ ]:
# Definindo as variaveis exogenas do conjunto de treino

train_exog_sarimax1 = training_set_sarimax1.iloc[:, 1:]
train_exog_sarimax1

In [ ]:
# Definindo as variaveis exogenas do conjunto de teste

test_exog_sarimax1 = test_set_sarimax1.iloc[:, 1:]
test_exog_sarimax1

In [ ]:
# Aplicando o modelo

model_sarimax1 = auto_arima(y = training_set_sarimax1['Compensada'],
                           X = train_exog_sarimax1,
                           m = 1,                   # Período anual: 12 meses (ajustar de acordo com a frequência escolhida anteriormente)
                           seasonal = True,
                           stepwise = False)

In [ ]:
model_sarimax1.summary()

In [ ]:
predictions_sarimax1 = pd.Series(model_sarimax1.predict(n_periods = 4, X = test_exog_sarimax1)).rename('SARIMAX')
predictions_sarimax1.index = test_set_sarimax1.index

In [ ]:
fig3 = go.Figure()

# Configurando o gráfico de linhas
fig3.add_trace(go.Scatter(x = test_set_sarimax1.index, y = test_set_sarimax1['Compensada'], mode = 'lines + markers', name = 'Compensada', marker_color = '#266B6E'))
fig3.add_trace(go.Scatter(x = predictions_sarimax1.index, y = predictions_sarimax1, mode = 'lines + markers', name = 'Predição Sarimax', marker_color = '#FF007D'))


# Configurando o título do gráfico
fig3.update_layout(title_text = f'Energia Compensada - 1',
                  title_font_size = 30,
                  title_x = 0.5)

# Configurando o título dos eixos
fig3.update_layout(xaxis_title_font_size = 18,
                  yaxis_title = 'Quantidade de Energia',
                  yaxis_title_font_size = 18,
                  showlegend = True,
                  legend_title = 'Legenda')

# Exibindo as grades de fundo
fig3.update_xaxes(showgrid = True)

# Plotando o gráfico
fig3.show()

In [ ]:
# Obtendo metricas de avaliacao

erros_sm1 = pd.DataFrame({'Métricas avaliadas': ['MAE', 'MAPE', 'WMAPE', 'MSE', 'RMSE'],
                        'Valores obtidos': [round(mean_absolute_error(test_set_sarimax1['Compensada'], predictions_sarimax1), 2),
                                            round(mean_absolute_percentage_error(test_set_sarimax1['Compensada'], predictions_sarimax1) * 100, 2),
                                            round(mean_absolute_percentage_error(test_set_sarimax1['Compensada'], predictions_sarimax1, sample_weight = test_set_sarimax1['Compensada']) * 100, 2),
                                            round(mean_squared_error(test_set_sarimax1['Compensada'], predictions_sarimax1, squared = True), 2),
                                            round(mean_squared_error(test_set_sarimax1['Compensada'], predictions_sarimax1, squared = False), 2)]},
                        index = range(0, 5))

erros_sm1

### <b>2.1.4. TensorFlow</b> <a class = "anchor" id = "chapter_2_1_4"></a>

In [ ]:
# Copia do dataset principal

usina1_tensorflow = usina1.copy(deep = True)

In [ ]:
# Definindo a frequencia dos dados

usina1_tensorflow = usina1_tensorflow.asfreq('MS')

In [ ]:
# Definindo o conjunto de treino (70%)

training_set_tensorflow1 = usina1_tensorflow.iloc[:-4, :]
training_set_tensorflow1

In [ ]:
# Definindo o conjunto de teste (30%)

test_set_tensorflow1 = usina1_tensorflow.iloc[-4:, :]
test_set_tensorflow1

In [ ]:
# Regressores

exog_tensorflow1 = np.asmatrix(usina1_tensorflow.iloc[:, 1:].astype(np.float64))
exog_tensorflow1

In [ ]:
# Aplicando o modelo

regressors_tensorflow1 = tfp.sts.LinearRegression(design_matrix = exog_tensorflow1, name = 'regressors')

In [ ]:
# Sazonalidade

Compensada_tensorflow1 = training_set_tensorflow1['Compensada'].astype(np.float64)
Compensada_tensorflow1

In [ ]:
monthly_effect_tensorflow1 = tfp.sts.Seasonal(num_seasons = 2,
                                             num_steps_per_season = np.array([[4]]),
                                             observed_time_series = Compensada_tensorflow1,
                                             name = 'month_effect')

In [ ]:
# Definindo a variavel autoregressive

autoregressive_tensorflow1 = tfp.sts.Autoregressive(order = 1, observed_time_series = Compensada_tensorflow1, name = 'autoregressive')

In [ ]:
# Definindo a variavel trend

trend_tensorflow1 = tfp.sts.LocalLinearTrend(observed_time_series = Compensada_tensorflow1, name = 'trend')

In [ ]:
# Aplicando o modelo novamente

model_tensorflow1 = tfp.sts.Sum([regressors_tensorflow1,
                                monthly_effect_tensorflow1,
                                autoregressive_tensorflow1,
                                trend_tensorflow1],
                                observed_time_series = Compensada_tensorflow1)

In [ ]:
samples_tensorflow1, kernel_results_tensorflow1 = tfp.sts.fit_with_hmc(model = model_tensorflow1,
                                                                     observed_time_series = Compensada_tensorflow1,
                                                                     num_results = 100,
                                                                     num_warmup_steps = 50,
                                                                     num_leapfrog_steps = 15,
                                                                     num_variational_steps = 150,
                                                                     seed = 1502)

In [ ]:
forecast_tensorflow1 = tfp.sts.forecast(model = model_tensorflow1,
                                       observed_time_series = Compensada_tensorflow1,
                                       parameter_samples = samples_tensorflow1,
                                       num_steps_forecast = len(test_set_tensorflow1))

In [ ]:
predictions_tensorflow1 = pd.Series(forecast_tensorflow1.mean()[:, 0], name = 'TFP')
predictions_tensorflow1.index = test_set_tensorflow1.index

In [ ]:
fig4 = go.Figure()

# Configurando o grafico de linhas
fig4.add_trace(go.Scatter(x = test_set_tensorflow1.index, y = test_set_tensorflow1['Compensada'], mode = 'lines + markers', name = 'Compensada', marker_color = '#266B6E'))
fig4.add_trace(go.Scatter(x = predictions_tensorflow1.index, y = predictions_tensorflow1, mode = 'lines + markers', name = 'Predição Tensorflow', marker_color = '#FF007D'))

# Configurando o titulo do grafico
fig4.update_layout(title_text = f'Energia Compensada - 1',
                  title_font_size = 30,
                  title_x = 0.5)

# Configurando o titulo dos eixos
fig4.update_layout(xaxis_title_font_size = 18,
                  yaxis_title = 'Quantidade de Energia',
                  yaxis_title_font_size = 18,
                  showlegend = True,
                  legend_title = 'Legenda')

# Exibindo as grades de fundo
fig4.update_xaxes(showgrid = True)

# Plotando o grafico
fig4.show()

In [ ]:
# Obtendo metricas de avaliacao

erros_tf1 = pd.DataFrame({'Métricas avaliadas': ['MAE', 'MAPE', 'WMAPE', 'MSE', 'RMSE'],
                                 'Valores obtidos': [round(mean_absolute_error(test_set_tensorflow1['Compensada'], predictions_tensorflow1), 2),
                                                     round(mean_absolute_percentage_error(test_set_tensorflow1['Compensada'], predictions_tensorflow1) * 100, 2),
                                                     round(mean_absolute_percentage_error(test_set_tensorflow1['Compensada'], predictions_tensorflow1, sample_weight = test_set_tensorflow1['Compensada']) * 100, 2),
                                                     round(mean_squared_error(test_set_tensorflow1['Compensada'], predictions_tensorflow1, squared = True), 2),
                                                     round(mean_squared_error(test_set_tensorflow1['Compensada'], predictions_tensorflow1, squared = False), 2)]},
                                index = range(0, 5))

erros_tf1

### <b>Organizando os dados para os modelos seguintes</b>

In [ ]:
usina1_modelos = usina1.copy(deep = True)

In [ ]:
# Dividindo os dados em conjuntos de treino e teste a partir de um corte temporal

df_train1 = usina1_modelos.loc[usina1_modelos.index < '2023-03-01']
df_test1 = usina1_modelos.loc[usina1_modelos.index >= '2023-03-01']

In [ ]:
# Separando os conjuntos X e Y de acordo com as colunas de dados

X_train1 = df_train1[['Acumulado', 'Energia_Ativa']]

y_train1 = df_train1['Compensada']
y_train1 = np.array(y_train1).reshape(-1, 1)

X_test1 = df_test1[['Acumulado', 'Energia_Ativa']]

y_test1 = df_test1['Compensada']
y_test1 = np.array(y_test1).reshape(-1, 1)

### <b>2.1.5 XGBoost </b> <a class = "anchor" id = "chapter_2_1_5"></a>

In [ ]:
# Instanciando o modelo
modelo_xgb = xgb.XGBRegressor(random_state = 0, max_depth=2)

# Ajustando o modelo
modelo_xgb1 = modelo_xgb.fit(X_train1, y_train1)

# Fazendo as predicoes
predicao_xgb1 = modelo_xgb1.predict(X_test1)

print('XGBoost')
# r2 dos dados de treino
print('R² do treino = {}'.format(modelo_xgb1.score(X_train1, y_train1)))

# r2 dos dados de teste
print('R² do teste = %s' % metrics.r2_score(y_test1, predicao_xgb1))

# Erro medio quadratico dos dados de teste e preditos
print('Raiz quadrada do erro médio quadrático dos resultados preditos = {}'.format(np.sqrt(mean_squared_error(y_test1, predicao_xgb1))))

# Erro medio absoluto dos dados de teste e treino
print('Erro médio absoluto dos resultados preditos = {}'.format(mean_absolute_error(y_test1, predicao_xgb1).round(2)))

In [ ]:
# Verificando quais variaveis impactam mais o modelo

fi1 = pd.DataFrame(data=modelo_xgb1.feature_importances_,
             index=modelo_xgb1.feature_names_in_,
             columns=['importância'])
fi1.sort_values('importância').plot(kind='barh', title='Importância das Variáveis - 1')
plt.show()

In [ ]:
# Fazendo um gridsearch para encontrar os melhores parametros

# Instanciando o modelo
modelo_xgb = xgb.XGBRegressor(random_state = 0)

# Definindo o espaco de busca dos hiperparametros
param_grid = {'learning_rate': np.linspace(0.001, 0.1, num = 20), 'n_estimators': np.arange(100, 200, 10),
               'max_depth': [2, 3, 4, 5]}

# Chamando o griddsearch e faz a busca aleatoria de acordo com os hiperparametros fornecidos acima
grid_search1 =  GridSearchCV(modelo_xgb, param_grid, scoring='neg_mean_squared_error', cv = 5)
grid_search1.fit(X_train1, y_train1)

# Acessando os melhores hiperparametros encontrados
grid_search1.best_params_


In [ ]:
# Rodando o modelo novamente, agora com os novos hiperparametros encontrados

# Instanciando o modelo
modelo_xgb = xgb.XGBRegressor(random_state = 0, n_estimators = 180, max_depth = 2, learning_rate = 0.079)

# Ajustando o modelo
modelo_xgb1 = modelo_xgb.fit(X_train1, y_train1)

# Fazendo as predicoes
predicao_xgb1 = modelo_xgb1.predict(X_test1)

print('XGBoost')
# r2 dos dados de treino
print('R² do treino = {}'.format(modelo_xgb1.score(X_train1, y_train1).round(2)))

# r2 dos dados de teste
print('R² do teste = %s' % metrics.r2_score(y_test1, predicao_xgb1).round(2))

# Erro quadratico medio dos dados de teste e preditos
print('Raiz quadrada do erro quadrático médio dos resultados preditos = {}'.format(np.sqrt(mean_squared_error(y_test1, predicao_xgb1)).round(2)))

# Erro medio absoluto dos dados de teste e treino
print('Erro médio absoluto dos resultados preditos = {}'.format(mean_absolute_error(y_test1, predicao_xgb1).round(2)))

In [ ]:
predictions_xgb1 = pd.DataFrame(y_test1, columns=['Compensada'])
predictions_xgb1.index = X_test1.index

predicao_xgb1 = pd.DataFrame(predicao_xgb1)
predicao_xgb1.index = X_test1.index

predictions_xgb1['Predição XGB'] = predicao_xgb1

In [ ]:
fig5 = go.Figure()

# Configurando o grafico de linhas
fig5.add_trace(go.Scatter(x = predictions_xgb1.index, y = predictions_xgb1['Compensada'], mode = 'lines + markers', name = 'Compensada', marker_color = '#266B6E'))
fig5.add_trace(go.Scatter(x = predictions_xgb1.index, y = predictions_xgb1['Predição XGB'], mode = 'lines + markers', name = 'Predição XGBoost', marker_color = '#FF007D'))

# Configurando o titulo do grafico
fig5.update_layout(title_text = f'Energia Compensada - 1',
                  title_font_size = 30,
                  title_x = 0.5)

# Configurando o titulo dos eixos
fig5.update_layout(xaxis_title_font_size = 18,
                  yaxis_title = 'Quantidade de Energia',
                  yaxis_title_font_size = 18,
                  showlegend = True,
                  legend_title = 'Legenda')

# Exibindo as grades de fundo
fig5.update_xaxes(showgrid = True)

# Plotando o grafico
fig5.show()

In [ ]:
# Obtendo metricas de avaliacao

erros_xgb1 = pd.DataFrame({'Métricas avaliadas': ['MAE', 'MAPE', 'WMAPE', 'MSE', 'RMSE'],
                          'Valores obtidos': [round(mean_absolute_error(y_test1, predicao_xgb1), 2),
                                              round(mean_absolute_percentage_error(y_test1, predicao_xgb1) * 100, 2),
                                              round(mean_absolute_percentage_error(y_test1, predicao_xgb1, sample_weight = y_test1) * 100, 2),
                                              round(mean_squared_error(y_test1, predicao_xgb1, squared = True), 2),
                                              round(mean_squared_error(y_test1, predicao_xgb1, squared = False), 2)]},
                        index = range(0, 5))

erros_xgb1

### <b>2.1.6 MLP </b> <a class = "anchor" id = "chapter_2_1_6"></a>

In [ ]:
# Instanciando o modelo
nnet = MLPRegressor(random_state = 0)

# Ajustando o modelo
modelo_nnet1 = nnet.fit(X_train1, y_train1)

# Fazendo as predições
predicao_nnet1 = modelo_nnet1.predict(X_test1)

print('MLP')
# r2 dos dados de treino
print('R² do treino = {}'.format(modelo_nnet1.score(X_train1, y_train1).round(2)))

# r2 dos dados de teste
print('R² do teste = %s' % metrics.r2_score(y_test1, predicao_nnet1).round(2))

In [ ]:
predictions_mlp1 = pd.DataFrame(y_test1, columns=['Compensada'])
predictions_mlp1.index = X_test1.index

predicao_nnet1 = pd.DataFrame(predicao_nnet1)
predicao_nnet1.index = X_test1.index

predictions_mlp1['Predição MLP'] = predicao_xgb1

In [ ]:
fig6 = go.Figure()

# Configurando o grafico de linhas
fig6.add_trace(go.Scatter(x = predictions_mlp1.index, y = predictions_mlp1['Compensada'], mode = 'lines + markers', name = 'Compensada', marker_color = '#266B6E'))
fig6.add_trace(go.Scatter(x = predictions_mlp1.index, y = predictions_mlp1['Predição MLP'], mode = 'lines + markers', name = 'Predição MLP', marker_color = '#FF007D'))

# Configurando o titulo do grafico
fig6.update_layout(title_text = f'Energia Compensada - 1',
                  title_font_size = 30,
                  title_x = 0.5)

# Configurando o titulo dos eixos
fig6.update_layout(xaxis_title_font_size = 18,
                  yaxis_title = 'Quantidade de Energia',
                  yaxis_title_font_size = 18,
                  showlegend = True,
                  legend_title = 'Legenda')

# Exibindo as grades de fundo
fig6.update_xaxes(showgrid = True)

# Plotando o grafico
fig6.show()

In [ ]:
# Obtendo metricas de avaliacao

erros_mlp1 = pd.DataFrame({'Métricas avaliadas': ['MAE', 'MAPE', 'WMAPE', 'MSE', 'RMSE'],
                          'Valores obtidos': [round(mean_absolute_error(y_test1, predicao_nnet1), 2),
                                              round(mean_absolute_percentage_error(y_test1, predicao_nnet1) * 100, 2),
                                              round(mean_absolute_percentage_error(y_test1, predicao_nnet1, sample_weight = y_test1) * 100, 2),
                                              round(mean_squared_error(y_test1, predicao_nnet1, squared = True), 2),
                                              round(mean_squared_error(y_test1, predicao_nnet1, squared = False), 2)]},
                        index = range(0, 5))

erros_mlp1

### <b>2.1.7 Random Forest </b> <a class = "anchor" id = "chapter_2_1_7"></a>

In [ ]:
# Instanciando o modelo
rf = RandomForestRegressor(random_state = 0)

# Ajustando o modelo
model_rf1 = rf.fit(X_train1, y_train1)

# Fazendo as predicoes
predicao_rf1 = model_rf1.predict(X_test1)

print('\n')

print('RANDOM FOREST')
# r2 dos dados de treino
print('R² do treino = {}'.format(model_rf1.score(X_train1, y_train1).round(2)))

# r2 dos dados de teste
print('R² do teste = %s' % metrics.r2_score(y_test1, predicao_rf1).round(2))

In [ ]:
predictions_rf1 = pd.DataFrame(y_test1, columns=['Compensada'])
predictions_rf1.index = X_test1.index

predicao_rf1 = pd.DataFrame(predicao_rf1)
predicao_rf1.index = X_test1.index

predictions_rf1['Predição Random Forest'] = predicao_rf1

In [ ]:
fig7 = go.Figure()

# Configurando o grafico de linhas
fig7.add_trace(go.Scatter(x = predictions_rf1.index, y = predictions_rf1['Compensada'], mode = 'lines + markers', name = 'Compensada', marker_color = '#266B6E'))
fig7.add_trace(go.Scatter(x = predictions_rf1.index, y = predictions_rf1['Predição Random Forest'], mode = 'lines + markers', name = 'Predição Random Forest', marker_color = '#FF007D'))

# Configurando o titulo do grafico
fig7.update_layout(title_text = f'Energia Compensada - 1',
                  title_font_size = 30,
                  title_x = 0.5)

# Configurando o titulo dos eixos
fig7.update_layout(xaxis_title_font_size = 18,
                  yaxis_title = 'Quantidade de Energia',
                  yaxis_title_font_size = 18,
                  showlegend = True,
                  legend_title = 'Legenda')

# Exibindo as grades de fundo
fig7.update_xaxes(showgrid = True)

# Plotando o grafico
fig7.show()

In [ ]:
# Obtendo metricas de avaliacao

erros_rf1 = pd.DataFrame({'Métricas avaliadas': ['MAE', 'MAPE', 'WMAPE', 'MSE', 'RMSE'],
                          'Valores obtidos': [round(mean_absolute_error(y_test1, predicao_rf1), 2),
                                              round(mean_absolute_percentage_error(y_test1, predicao_rf1) * 100, 2),
                                              round(mean_absolute_percentage_error(y_test1, predicao_rf1, sample_weight = y_test1) * 100, 2),
                                              round(mean_squared_error(y_test1, predicao_rf1, squared = True), 2),
                                              round(mean_squared_error(y_test1, predicao_rf1, squared = False), 2)]},
                        index = range(0, 5))

erros_rf1

### <b>2.1.8 Regressão Linear </b> <a class = "anchor" id = "chapter_2_1_8"></a>

In [ ]:
# Regressao Linear

# Instanciando  o modelo 
rl = LinearRegression()

# Ajustando o modelo
modelo_rl1 = rl.fit(X_train1, y_train1)

# Fazendo as predicoes
predicao_rl1 = modelo_rl1.predict(X_test1)

print('REGRESSÃO LINEAR')
# r2 dos dados de treino
print('R² do treino = {}'.format(modelo_rl1.score(X_train1, y_train1).round(2)))

# r2 dos dados de teste
print('R² do teste = %s' % metrics.r2_score(y_test1, predicao_rl1).round(2))


In [ ]:
predictions_rl1 = pd.DataFrame(y_test1, columns=['Compensada'])
predictions_rl1.index = X_test1.index

predicao_rl1 = pd.DataFrame(predicao_rl1)
predicao_rl1.index = X_test1.index

predictions_rl1['Predição Regressão Linear'] = predicao_rl1

In [ ]:
fig8 = go.Figure()

# Configurando o grafico de linhas
fig8.add_trace(go.Scatter(x = predictions_rl1.index, y = predictions_rl1['Compensada'], mode = 'lines + markers', name = 'Compensada', marker_color = '#266B6E'))
fig8.add_trace(go.Scatter(x = predictions_rl1.index, y = predictions_rl1['Predição Regressão Linear'], mode = 'lines + markers', name = 'Predição Regressão Linear', marker_color = '#FF007D'))

# Configurando o titulo do grafico
fig8.update_layout(title_text = f'Energia Compensada - 1',
                  title_font_size = 30,
                  title_x = 0.5)

# Configurando o titulo dos eixos
fig8.update_layout(xaxis_title_font_size = 18,
                  yaxis_title = 'Quantidade de Energia',
                  yaxis_title_font_size = 18,
                  showlegend = True,
                  legend_title = 'Legenda')

# Exibindo as grades de fundo
fig8.update_xaxes(showgrid = True)

# Plotando o grafico
fig8.show()

In [ ]:
# Obtendo metricas de avaliacao

erros_rl1 = pd.DataFrame({'Métricas avaliadas': ['MAE', 'MAPE', 'WMAPE', 'MSE', 'RMSE'],
                          'Valores obtidos': [round(mean_absolute_error(y_test1, predicao_rl1), 2),
                                              round(mean_absolute_percentage_error(y_test1, predicao_rl1) * 100, 2),
                                              round(mean_absolute_percentage_error(y_test1, predicao_rl1, sample_weight = y_test1) * 100, 2),
                                              round(mean_squared_error(y_test1, predicao_rl1, squared = True), 2),
                                              round(mean_squared_error(y_test1, predicao_rl1, squared = False), 2)]},
                        index = range(0, 5))

erros_rl1

### <b>2.2. Usina  2</b> <a class = "anchor" id = "chapter_2_2"></a>

In [ ]:
usina2 = df[df['Usina'] == ' 2'].groupby('Competência').agg({'Compensada':'sum', 'Acumulado':'sum', 
                                                             'Energia_Ativa':'sum'})

usina2

In [ ]:
# Descricao estatistica dos dados

usina2.describe()

In [ ]:
fig9 = go.Figure()

# Configurando o gráfico de barras
fig9.add_trace(go.Bar(x = usina2.index,
                     y = usina2['Compensada'],
                     text = usina2['Compensada'],
                     name = 'Gráfico de barras',
                     textposition = 'inside',
                     textfont_size = 12,
                     marker_color = '#266B6E'))

# Configurando o gráfico de linhas
fig9.add_trace(go.Scatter(x = usina2.index,
                         y = usina2['Compensada'],
                         name = 'Gráfico de linhas',
                         mode = 'lines + markers',
                         marker_color = 'red'))


# Configurando o título do gráfico
fig9.update_layout(title_text = f'Energia Compensada -  2',
                  title_font_size = 30,
                  title_x = 0.5)

# Configurando o título dos eixos
fig9.update_layout(yaxis_title = 'Quantidade de energia',
                  yaxis_title_font_size = 18,
                  xaxis_title = 'Meses',
                  showlegend = False)

# Exibindo as grades de fundo
fig9.update_xaxes(showgrid = True)

# Plotando o gráfico
fig9.show()

### <b>2.2.1. LSTM</b> <a class = "anchor" id = "chapter_2_2_1"></a>

In [ ]:
# Copia do dataset principal

usina2_lstm = usina2.copy(deep = True)

In [ ]:
# Separando quais dados serao utilizados para rodar o modelo

usina2_lstm = usina2_lstm.loc[:,['Compensada']]
usina2_lstm

In [ ]:
# Definindo a frequencia dos dados

usina2_lstm = usina2_lstm.asfreq('MS')

In [ ]:
# Normalizando os dados

scaler_lstm = MinMaxScaler(feature_range = (0, 1))
data_lstm2 = scaler_lstm.fit_transform(usina2_lstm)

In [ ]:
# Dividindo os dados em conjuntos de treino e teste

train_size_lstm2 = int(len(data_lstm2) * 0.7)               #70% para treino e 30% para teste
training_set_lstm2, test_set_lstm2 = data_lstm2[:train_size_lstm2], data_lstm2[train_size_lstm2:]

In [ ]:
# Atribuindo valor a janela deslizante

lookback_window = 1

In [ ]:
# Criando a funcao de janela deslizante

def create_dataset(data, lookback_window):
    X, y = [], []

    for i in range(len(data) - lookback_window):
        X.append(data[i:(i + lookback_window), :])
        y.append(data[i + lookback_window, 0])

    return np.array(X), np.array(y)

In [ ]:
# Ajustando os conjuntos de treino e teste

X_train_lstm2, y_train_lstm2 = create_dataset(training_set_lstm2, lookback_window)
X_test_lstm2, y_test_lstm2 = create_dataset(test_set_lstm2, lookback_window)

In [ ]:
# Ajustando parametros do modelo LSTM

best_model_lstm = Sequential()
best_model_lstm.add(LSTM(4, input_shape = (lookback_window, 1)))
best_model_lstm.add(Dense(1))
best_model_lstm.compile(loss = 'mean_squared_error', optimizer = 'adam', run_eagerly=True)

history_lstm2 = best_model_lstm.fit(X_train_lstm2,
                                   y_train_lstm2,
                                   epochs = 170,
                                   batch_size = 1,
                                   validation_split = 0.2,
                                   verbose = 2)

In [ ]:
# Rodando modelo para conjunto de teste

y_pred_lstm2 = best_model_lstm.predict(X_test_lstm2)

y_pred_lstm2 = scaler_lstm.inverse_transform(y_pred_lstm2)
y_test_lstm2 = scaler_lstm.inverse_transform(y_test_lstm2.reshape(-1, 1))

In [ ]:
# Criando dataframe com as predicoes

timestamp2 = pd.DataFrame(usina2_lstm.index[-len(X_test_lstm2):])

y_test_df_lstm2 = pd.concat([timestamp2, pd.DataFrame(y_test_lstm2)], axis = 1).rename(columns = {0: 'Compensada'}).dropna()
y_pred_df_lstm2 = pd.concat([timestamp2, pd.DataFrame(y_pred_lstm2)], axis = 1).rename(columns = {0: 'Compensada_predito'}).dropna()


# Ajuste nas datas para corresponderem as datas reais dos registros de teste

y_test_df_lstm2.index = usina2_lstm.index[-len(X_test_lstm2):]
y_pred_df_lstm2.index = usina2_lstm.index[-len(X_test_lstm2):]

In [ ]:
fig10 = go.Figure()

# Configurando o grafico de linhas
fig10.add_trace(go.Scatter(x = y_test_df_lstm2['Competência'], y = y_test_df_lstm2['Compensada'], mode = 'lines + markers', name = 'Compensada', marker_color = '#266B6E'))
fig10.add_trace(go.Scatter(x = y_pred_df_lstm2['Competência'], y = y_pred_df_lstm2['Compensada_predito'], mode = 'lines + markers', name = 'Predição LSTM', marker_color = '#FF007D'))

# Configurando o titulo do grafico
fig10.update_layout(title_text = f'Energia Compensada -  2',
                  title_font_size = 30,
                  title_x = 0.5)

# Configurando o titulo dos eixos
fig10.update_layout(xaxis_title_font_size = 18,
                  yaxis_title = 'Quantidade de Energia',
                  yaxis_title_font_size = 18,
                  showlegend = True,
                  legend_title = 'Legenda')

# Exibindo as grades de fundo
fig10.update_xaxes(showgrid = True)

# Plotando o grafico
fig10.show()

In [ ]:
# Obtendo metricas de avaliacao

erros_lstm2 = pd.DataFrame({'Métricas avaliadas': ['MAE', 'MAPE', 'WMAPE', 'MSE', 'RMSE'],
                                'Valores obtidos': [round(mean_absolute_error(y_test_lstm2, y_pred_lstm2), 2),
                                                    round(mean_absolute_percentage_error(y_test_lstm2, y_pred_lstm2) * 100, 2),
                                                    round(mean_absolute_percentage_error(y_test_lstm2, y_pred_lstm2, sample_weight = y_test_lstm2) * 100, 2),
                                                    round(mean_squared_error(y_test_lstm2, y_pred_lstm2, squared = True), 2),
                                                    round(mean_squared_error(y_test_lstm2, y_pred_lstm2, squared = False), 2)]},
                            index = range(0, 5))

erros_lstm2

### <b>2.2.2. Holt-Winters</b> <a class = "anchor" id = "chapter_2_2_2"></a>

In [ ]:
# Copia do dataset principal

usina2_holtw = usina2.copy(deep = True)

In [ ]:
# Separando quais dados serao utilizados para rodar o modelo

usina2_holtw = usina2_holtw.loc[:,['Compensada']]

In [ ]:
# Definindo a frequencia dos dados

usina2_holtw = usina2_holtw.asfreq('MS')

In [ ]:
# Dividindo dados em conjunto de treino (70%)

training_set_holtwinters2 = usina2_holtw.iloc[:-4, :]
training_set_holtwinters2

In [ ]:
# Dividindo dados em conjunto de teste (30%)

test_set_holtwinters2 = usina2_holtw.iloc[-4:, :]
test_set_holtwinters2

In [ ]:
# Aplicando o modelo

model_holtwinters2 = ExponentialSmoothing(endog = training_set_holtwinters2['Compensada'],     
                                         trend = 'add',                             
                                         seasonal = 'add',
                                         seasonal_periods = 4).fit()         

In [ ]:
predictions_holtwinters2 = pd.Series(model_holtwinters2.forecast(steps = len(test_set_holtwinters2))).rename('HW')
predictions_holtwinters2.index = test_set_holtwinters2.index

In [ ]:
fig11 = go.Figure()

# Configurando o grafico de linhas
fig11.add_trace(go.Scatter(x = test_set_holtwinters2.index, y = test_set_holtwinters2['Compensada'], mode = 'lines + markers', name = 'Compensada', marker_color = '#266B6E'))
fig11.add_trace(go.Scatter(x = predictions_holtwinters2.index, y = predictions_holtwinters2, mode = 'lines + markers', name = 'Predição Holt-Winters', marker_color = '#FF007D'))

# Configurando o titulo do grafico
fig11.update_layout(title_text = f'Energia Compensada -  2',
                  title_font_size = 30,
                  title_x = 0.5)

# Configurando o titulo dos eixos
fig11.update_layout(xaxis_title_font_size = 18,
                  yaxis_title = 'Quantidade de Energia',
                  yaxis_title_font_size = 18,
                  showlegend = True,
                  legend_title = 'Legenda')

# Exibindo as grades de fundo
fig11.update_xaxes(showgrid = True)

# Plotando o grafico
fig11.show()

In [ ]:
# Obtendo metricas de avaliacao

erros_hw2 = pd.DataFrame({'Métricas avaliadas': ['MAE', 'MAPE', 'WMAPE', 'MSE', 'RMSE'],
                        'Valores obtidos': [round(mean_absolute_error(test_set_holtwinters2['Compensada'], predictions_holtwinters2), 2),
                                            round(mean_absolute_percentage_error(test_set_holtwinters2['Compensada'], predictions_holtwinters2) * 100, 2),
                                            round(mean_absolute_percentage_error(test_set_holtwinters2['Compensada'], predictions_holtwinters2, sample_weight = test_set_holtwinters2['Compensada']) * 100, 2),
                                            round(mean_squared_error(test_set_holtwinters2['Compensada'], predictions_holtwinters2, squared = True), 2),
                                            round(mean_squared_error(test_set_holtwinters2['Compensada'], predictions_holtwinters2, squared = False), 2)]},
                        index = range(0, 5))

erros_hw2

### <b>2.2.3 SARIMAX </b> <a class = "anchor" id = "chapter_2_2_3"></a>

In [ ]:
# Copia do dataset principal

usina2_sarimax = usina2.copy(deep = True)

In [ ]:
# Definindo a frequencia dos dados

usina2_sarimax = usina2_sarimax.asfreq('MS')

In [ ]:
# Definindo o conjunto de treino (70%)

training_set_sarimax2 = usina2_sarimax.iloc[:-4, :]
training_set_sarimax2

In [ ]:
# Definindo o conjunto de teste (30%)

test_set_sarimax2 = usina2_sarimax.iloc[-4:, :]
test_set_sarimax2

In [ ]:
# Definindo as variaveis exogenas de treino

train_exog_sarimax2 = training_set_sarimax2.iloc[:, 1:]
train_exog_sarimax2

In [ ]:
# Definindo as variaveis exogenas de teste

test_exog_sarimax2 = test_set_sarimax2.iloc[:, 1:]
test_exog_sarimax2

In [ ]:
# Aplicando o modelo

model_sarimax2 = auto_arima(y = training_set_sarimax2['Compensada'],
                           X = train_exog_sarimax2,
                           m = 1,                   # Período anual: 12 meses (ajustar de acordo com a frequência escolhida anteriormente)
                           seasonal = True,
                           stepwise = False)

In [ ]:
model_sarimax2.summary()

In [ ]:
predictions_sarimax2 = pd.Series(model_sarimax2.predict(n_periods = 4, X = test_exog_sarimax2)).rename('SARIMAX')
predictions_sarimax2.index = test_set_sarimax2.index

In [ ]:
fig12 = go.Figure()

# Configurando o grafico de linhas
fig12.add_trace(go.Scatter(x = test_set_sarimax2.index, y = test_set_sarimax2['Compensada'], mode = 'lines + markers', name = 'Compensada', marker_color = '#266B6E'))
fig12.add_trace(go.Scatter(x = predictions_sarimax2.index, y = predictions_sarimax2, mode = 'lines + markers', name = 'Predição Sarimax', marker_color = '#FF007D'))


# Configurando o titulo do grafico
fig12.update_layout(title_text = f'Energia Compensada -  2',
                  title_font_size = 30,
                  title_x = 0.5)

# Configurando o titulo dos eixos
fig12.update_layout(xaxis_title_font_size = 18,
                  yaxis_title = 'Quantidade de Energia',
                  yaxis_title_font_size = 18,
                  showlegend = True,
                  legend_title = 'Legenda')

# Exibindo as grades de fundo
fig12.update_xaxes(showgrid = True)

# Plotando o grafico
fig12.show()

In [ ]:
# Obtendo metricas de avaliacao

erros_sm2 = pd.DataFrame({'Métricas avaliadas': ['MAE', 'MAPE', 'WMAPE', 'MSE', 'RMSE'],
                        'Valores obtidos': [round(mean_absolute_error(test_set_sarimax2['Compensada'], predictions_sarimax2), 2),
                                            round(mean_absolute_percentage_error(test_set_sarimax2['Compensada'], predictions_sarimax2) * 100, 2),
                                            round(mean_absolute_percentage_error(test_set_sarimax2['Compensada'], predictions_sarimax2, sample_weight = test_set_sarimax2['Compensada']) * 100, 2),
                                            round(mean_squared_error(test_set_sarimax2['Compensada'], predictions_sarimax2, squared = True), 2),
                                            round(mean_squared_error(test_set_sarimax2['Compensada'], predictions_sarimax2, squared = False), 2)]},
                        index = range(0, 5))

erros_sm2

### <b>2.2.4 TensorFlow </b> <a class = "anchor" id = "chapter_2_2_4"></a>

In [ ]:
# Copia do dataset principal

usina2_tensorflow = usina2.copy(deep = True)

In [ ]:
# Definindo a frequencia dos dados

usina2_tensorflow = usina2_tensorflow.asfreq('MS')

In [ ]:
# Definindo o conjunto de treino (70%)

training_set_tensorflow2 = usina2_tensorflow.iloc[:-4, :]
training_set_tensorflow2

In [ ]:
# Definindo o conjunto de teste (30%)

test_set_tensorflow2 = usina2_tensorflow.iloc[-4:, :]
test_set_tensorflow2

In [ ]:
# Regressores

exog_tensorflow2 = np.asmatrix(usina2_tensorflow.iloc[:, 1:].astype(np.float64))

In [ ]:
# Aplicando o modelo

regressors_tensorflow2 = tfp.sts.LinearRegression(design_matrix = exog_tensorflow2, name = 'regressors')

In [ ]:
# Sazonalidade

Compensada_tensorflow2 = training_set_tensorflow2['Compensada'].astype(np.float64)

In [ ]:
monthly_effect_tensorflow2 = tfp.sts.Seasonal(num_seasons = 4,
                                             num_steps_per_season = np.array([[4]]),
                                             observed_time_series = Compensada_tensorflow2,
                                             name = 'month_effect')

In [ ]:
# Definindo a variavel autoregressive

autoregressive_tensorflow2 = tfp.sts.Autoregressive(order = 1, observed_time_series = Compensada_tensorflow2, name = 'autoregressive')

In [ ]:
# Definindo a variavel trend

trend_tensorflow2 = tfp.sts.LocalLinearTrend(observed_time_series = Compensada_tensorflow2, name = 'trend')

In [ ]:
# Aplicando o modelo novamente

model_tensorflow2 = tfp.sts.Sum([regressors_tensorflow2,
                                monthly_effect_tensorflow2,
                                autoregressive_tensorflow2,
                                trend_tensorflow2],
                                observed_time_series = Compensada_tensorflow2)

In [ ]:
samples_tensorflow2, kernel_results_tensorflow2 = tfp.sts.fit_with_hmc(model = model_tensorflow2,
                                                                     observed_time_series = Compensada_tensorflow2,
                                                                     num_results = 100,
                                                                     num_warmup_steps = 50,
                                                                     num_leapfrog_steps = 15,
                                                                     num_variational_steps = 150,
                                                                     seed = 1502)

In [ ]:
forecast_tensorflow2 = tfp.sts.forecast(model = model_tensorflow2,
                                       observed_time_series = Compensada_tensorflow2,
                                       parameter_samples = samples_tensorflow2,
                                       num_steps_forecast = len(test_set_tensorflow2))

In [ ]:
predictions_tensorflow2 = pd.Series(forecast_tensorflow2.mean()[:, 0], name = 'TFP')
predictions_tensorflow2.index = test_set_tensorflow2.index

In [ ]:
fig13 = go.Figure()

# Configurando o gráfico de linhas
fig13.add_trace(go.Scatter(x = test_set_tensorflow2.index, y = test_set_tensorflow2['Compensada'], mode = 'lines + markers', name = 'Compensada', marker_color = '#266B6E'))
fig13.add_trace(go.Scatter(x = predictions_tensorflow2.index, y = predictions_tensorflow2, mode = 'lines + markers', name = 'Predição Tensorflow', marker_color = '#FF007D'))

# Configurando o título do gráfico
fig13.update_layout(title_text = f'Energia Compensada -  2',
                  title_font_size = 30,
                  title_x = 0.5)

# Configurando o título dos eixos
fig13.update_layout(xaxis_title_font_size = 18,
                  yaxis_title = 'Quantidade de Energia',
                  yaxis_title_font_size = 18,
                  showlegend = True,
                  legend_title = 'Legenda')

# Exibindo as grades de fundo
fig13.update_xaxes(showgrid = True)

# Plotando o gráfico
fig13.show()

In [ ]:
# Obtendo metricas de avaliacao

erros_tf2 = pd.DataFrame({'Métricas avaliadas': ['MAE', 'MAPE', 'WMAPE', 'MSE', 'RMSE'],
                                 'Valores obtidos': [round(mean_absolute_error(test_set_tensorflow2['Compensada'], predictions_tensorflow2), 2),
                                                     round(mean_absolute_percentage_error(test_set_tensorflow2['Compensada'], predictions_tensorflow2) * 100, 2),
                                                     round(mean_absolute_percentage_error(test_set_tensorflow2['Compensada'], predictions_tensorflow2, sample_weight = test_set_tensorflow2['Compensada']) * 100, 2),
                                                     round(mean_squared_error(test_set_tensorflow2['Compensada'], predictions_tensorflow2, squared = True), 2),
                                                     round(mean_squared_error(test_set_tensorflow2['Compensada'], predictions_tensorflow2, squared = False), 2)]},
                                index = range(0, 5))

erros_tf2

### <b>Organizando os dados para os modelos seguintes</b>

In [ ]:
usina2_modelos = usina2.copy(deep = True)
usina2_modelos

In [ ]:
# Dividindo os dados em conjuntos de treino e teste a partir de um corte temporal

df_train2 = usina2_modelos.loc[usina2_modelos.index < '2023-03-01']
df_test2 = usina2_modelos.loc[usina2_modelos.index >= '2023-03-01']

In [ ]:
# Separando os conjuntos X e Y de acordo com as colunas de dados

X_train2 = df_train2[['Acumulado', 'Energia_Ativa']]

y_train2 = df_train2['Compensada']
y_train2 = np.array(y_train2).reshape(-1, 1)

X_test2 = df_test2[['Acumulado', 'Energia_Ativa']]

y_test2 = df_test2['Compensada']
y_test2 = np.array(y_test2).reshape(-1, 1)

### <b>2.2.5 XGBoost </b> <a class = "anchor" id = "chapter_2_2_5"></a>

In [ ]:
# Instanciando o modelo
modelo_xgb = xgb.XGBRegressor(random_state = 0, max_depth=2)

# Ajustando o modelo
modelo_xgb2 = modelo_xgb.fit(X_train2, y_train2)

# Fazendo as predicoes
predicao_xgb2 = modelo_xgb2.predict(X_test2)

print('XGBoost')
# r2 dos dados de treino
print('R² do treino = {}'.format(modelo_xgb2.score(X_train2, y_train2)))

# r2 dos dados de teste
print('R² do teste = %s' % metrics.r2_score(y_test2, predicao_xgb2))

# Erro medio quadratico dos dados de teste e preditos
print('Raiz quadrada do erro médio quadrático dos resultados preditos = {}'.format(np.sqrt(mean_squared_error(y_test2, predicao_xgb2))))

# Erro medio absoluto dos dados de teste e treino
print('Erro médio absoluto dos resultados preditos = {}'.format(mean_absolute_error(y_test2, predicao_xgb2).round(2)))

In [ ]:
# Verificando quais variaveis impactam mais o modelo

fi2 = pd.DataFrame(data=modelo_xgb2.feature_importances_,
             index=modelo_xgb2.feature_names_in_,
             columns=['importância'])
fi2.sort_values('importância').plot(kind='barh', title='Importância das variáveis -  2')
plt.show()

In [ ]:
# Fazendo um gridsearch para encontrar os melhores parametros

# Intanciando o modelo
modelo_xgb = xgb.XGBRegressor(random_state = 0)

# Definindo o espaco de busca dos hiperparametros
param_grid = {'learning_rate': np.linspace(0.001, 0.1, num = 20), 'n_estimators': np.arange(100, 200, 10),
               'max_depth': [2, 3, 4, 5]}

# Chamando o griddsearch e faz a busca aleatoria de acordo com os hiperparametros fornecidos acima
grid_search2 =  GridSearchCV(modelo_xgb, param_grid, scoring='neg_mean_squared_error', cv = 5)
grid_search2.fit(X_train2, y_train2)

# Acessando os melhores hiperparametros encontrados
grid_search2.best_params_

In [ ]:
#Rodando o modelo novamente, agora com os novos parametros encontrados

# Instanciando o modelo
modelo_xgb = xgb.XGBRegressor(random_state = 0, n_estimators = 190, max_depth = 4, learning_rate = 0.084)

# Ajustando o modelo
modelo_xgb2 = modelo_xgb.fit(X_train2, y_train2)

# Fazenco as predicoes
predicao_xgb2 = modelo_xgb2.predict(X_test2)

print('XGBoost')
# r2 dos dados de treino
print('R² do treino = {}'.format(modelo_xgb2.score(X_train2, y_train2).round(2)))

# r2 dos dados de teste
print('R² do teste = %s' % metrics.r2_score(y_test2, predicao_xgb2).round(2))

# Erro quadratico medio dos dados de teste e preditos
print('Raiz quadrada do erro quadrático médio dos resultados preditos = {}'.format(np.sqrt(mean_squared_error(y_test2, predicao_xgb2)).round(2)))

# Erro medio absoluto dos dados de teste e treino
print('Erro médio absoluto dos resultados preditos = {}'.format(mean_absolute_error(y_test2, predicao_xgb2).round(2)))

In [ ]:
predictions_xgb2 = pd.DataFrame(y_test2, columns=['Compensada'])
predictions_xgb2.index = X_test2.index

predicao_xgb2 = pd.DataFrame(predicao_xgb2)
predicao_xgb2.index = X_test2.index

predictions_xgb2['Predição XGB'] = predicao_xgb2

In [ ]:
fig14 = go.Figure()

# Configurando o grafico de linhas
fig14.add_trace(go.Scatter(x = predictions_xgb2.index, y = predictions_xgb2['Compensada'], mode = 'lines + markers', name = 'Compensada', marker_color = '#266B6E'))
fig14.add_trace(go.Scatter(x = predictions_xgb2.index, y = predictions_xgb2['Predição XGB'], mode = 'lines + markers', name = 'Predição XGBoost', marker_color = '#FF007D'))

# Configurando o titulo do grafico
fig14.update_layout(title_text = f'Energia Compensada -  2',
                  title_font_size = 30,
                  title_x = 0.5)

# Configurando o titulo dos eixos
fig14.update_layout(xaxis_title_font_size = 18,
                  yaxis_title = 'Quantidade de Energia',
                  yaxis_title_font_size = 18,
                  showlegend = True,
                  legend_title = 'Legenda')

# Exibindo as grades de fundo
fig14.update_xaxes(showgrid = True)

# Plotando o grafico
fig14.show()

In [ ]:
erros_xgb2 = pd.DataFrame({'Métricas avaliadas': ['MAE', 'MAPE', 'WMAPE', 'MSE', 'RMSE'],
                          'Valores obtidos': [round(mean_absolute_error(y_test2, predicao_xgb2), 2),
                                              round(mean_absolute_percentage_error(y_test2, predicao_xgb2) * 100, 2),
                                              round(mean_absolute_percentage_error(y_test2, predicao_xgb2, sample_weight = y_test2) * 100, 2),
                                              round(mean_squared_error(y_test2, predicao_xgb2, squared = True), 2),
                                              round(mean_squared_error(y_test2, predicao_xgb2, squared = False), 2)]},
                        index = range(0, 5))

erros_xgb2

### <b>2.2.6 MLP </b> <a class = "anchor" id = "chapter_2_2_6"></a>

In [ ]:
# Instanciando o modelo
nnet = MLPRegressor(random_state = 0)

# Ajustando o modelo
modelo_nnet2 = nnet.fit(X_train2, y_train2)

# Fazendo as predicoes
predicao_nnet2 = modelo_nnet2.predict(X_test2)

print('MLP')
# r2 dos dados de treino
print('R² do treino = {}'.format(modelo_nnet2.score(X_train2, y_train2).round(2)))

# r2 dos dados de teste
print('R² do teste = %s' % metrics.r2_score(y_test2, predicao_nnet2).round(2))

In [ ]:
predictions_mlp2 = pd.DataFrame(y_test2, columns=['Compensada'])
predictions_mlp2.index = X_test2.index

predicao_nnet2 = pd.DataFrame(predicao_nnet2)
predicao_nnet2.index = X_test2.index

predictions_mlp2['Predição MLP'] = predicao_xgb2

In [ ]:
fig15 = go.Figure()

# Configurando o grafico de linhas
fig15.add_trace(go.Scatter(x = predictions_mlp2.index, y = predictions_mlp2['Compensada'], mode = 'lines + markers', name = 'Compensada', marker_color = '#266B6E'))
fig15.add_trace(go.Scatter(x = predictions_mlp2.index, y = predictions_mlp2['Predição MLP'], mode = 'lines + markers', name = 'Predição MLP', marker_color = '#FF007D'))

# Configurando o titulo do grafico
fig15.update_layout(title_text = f'Energia Compensada -  2',
                  title_font_size = 30,
                  title_x = 0.5)

# Configurando o titulo dos eixos
fig15.update_layout(xaxis_title_font_size = 18,
                  yaxis_title = 'Quantidade de Energia',
                  yaxis_title_font_size = 18,
                  showlegend = True,
                  legend_title = 'Legenda')

# Exibindo as grades de fundo
fig15.update_xaxes(showgrid = True)

# Plotando o grafico
fig15.show()

In [ ]:
erros_mlp2 = pd.DataFrame({'Métricas avaliadas': ['MAE', 'MAPE', 'WMAPE', 'MSE', 'RMSE'],
                          'Valores obtidos': [round(mean_absolute_error(y_test2, predicao_nnet2), 2),
                                              round(mean_absolute_percentage_error(y_test2, predicao_nnet2) * 100, 2),
                                              round(mean_absolute_percentage_error(y_test2, predicao_nnet2, sample_weight = y_test2) * 100, 2),
                                              round(mean_squared_error(y_test2, predicao_nnet2, squared = True), 2),
                                              round(mean_squared_error(y_test2, predicao_nnet2, squared = False), 2)]},
                        index = range(0, 5))

erros_mlp2

### <b>2.2.7 Random Forest </b> <a class = "anchor" id = "chapter_2_2_7"></a>

In [ ]:
# Instanciando o modelo
rf = RandomForestRegressor(random_state = 0)

# Ajustando o modelo
model_rf2 = rf.fit(X_train2, y_train2)

# Fazendo as predicoes
predicao_rf2 = model_rf2.predict(X_test2)

print('\n')

print('RANDOM FOREST')
# r2 dos dados de treino
print('R² do treino = {}'.format(model_rf2.score(X_train2, y_train2).round(2)))

# r2 dos dados de teste
print('R² do teste = %s' % metrics.r2_score(y_test2, predicao_rf2).round(2))

In [ ]:
predictions_rf2 = pd.DataFrame(y_test2, columns=['Compensada'])
predictions_rf2.index = X_test2.index

predicao_rf2 = pd.DataFrame(predicao_rf2)
predicao_rf2.index = X_test2.index

predictions_rf2['Predição Random Forest'] = predicao_rf1

In [ ]:
fig16 = go.Figure()

# Configurando o grafico de linhas
fig16.add_trace(go.Scatter(x = predictions_rf1.index, y = predictions_rf1['Compensada'], mode = 'lines + markers', name = 'Compensada', marker_color = '#266B6E'))
fig16.add_trace(go.Scatter(x = predictions_rf1.index, y = predictions_rf1['Predição Random Forest'], mode = 'lines + markers', name = 'Predição Random Forest', marker_color = '#FF007D'))

# Configurando o titulo do grafico
fig16.update_layout(title_text = f'Energia Compensada -  2',
                  title_font_size = 30,
                  title_x = 0.5)

# Configurando o titulo dos eixos
fig16.update_layout(xaxis_title_font_size = 18,
                  yaxis_title = 'Quantidade de Energia',
                  yaxis_title_font_size = 18,
                  showlegend = True,
                  legend_title = 'Legenda')

# Exibindo as grades de fundo
fig16.update_xaxes(showgrid = True)

# Plotando o grafico
fig16.show()

In [ ]:
erros_rf2 = pd.DataFrame({'Métricas avaliadas': ['MAE', 'MAPE', 'WMAPE', 'MSE', 'RMSE'],
                          'Valores obtidos': [round(mean_absolute_error(y_test2, predicao_rf2), 2),
                                              round(mean_absolute_percentage_error(y_test2, predicao_rf2) * 100, 2),
                                              round(mean_absolute_percentage_error(y_test2, predicao_rf2, sample_weight = y_test2) * 100, 2),
                                              round(mean_squared_error(y_test2, predicao_rf2, squared = True), 2),
                                              round(mean_squared_error(y_test2, predicao_rf2, squared = False), 2)]},
                        index = range(0, 5))

erros_rf2 

### <b>2.2.8 Regressão Linear </b> <a class = "anchor" id = "chapter_2_2_8"></a>

In [ ]:
# Instanciando o modelo 
rl = LinearRegression()

# Ajustando o modelo
modelo_rl2 = rl.fit(X_train2, y_train2)

# Fazendo as predicoes
predicao_rl2 = modelo_rl2.predict(X_test2)

print('REGRESSÃO LINEAR')
# r2 dos dados de treino
print('R² do treino = {}'.format(modelo_rl2.score(X_train2, y_train2).round(2)))

# r2 dos dados de teste
print('R² do teste = %s' % metrics.r2_score(y_test2, predicao_rl2).round(2))

In [ ]:
predictions_rl2 = pd.DataFrame(y_test2, columns=['Compensada'])
predictions_rl2.index = X_test2.index

predicao_rl2 = pd.DataFrame(predicao_rl2)
predicao_rl2.index = X_test2.index

predictions_rl2['Predição Regressão Linear'] = predicao_rl2

In [ ]:
fig17 = go.Figure()

# Configurando o grafico de linhas
fig17.add_trace(go.Scatter(x = predictions_rl2.index, y = predictions_rl2['Compensada'], mode = 'lines + markers', name = 'Compensada', marker_color = '#266B6E'))
fig17.add_trace(go.Scatter(x = predictions_rl2.index, y = predictions_rl2['Predição Regressão Linear'], mode = 'lines + markers', name = 'Predição Regressão Linear', marker_color = '#FF007D'))

# Configurando o titulo do grafico
fig17.update_layout(title_text = f'Energia Compensada -  2',
                  title_font_size = 30,
                  title_x = 0.5)

# Configurando o titulo dos eixos
fig17.update_layout(xaxis_title_font_size = 18,
                  yaxis_title = 'Quantidade de Energia',
                  yaxis_title_font_size = 18,
                  showlegend = True,
                  legend_title = 'Legenda')

# Exibindo as grades de fundo
fig17.update_xaxes(showgrid = True)

# Plotando o grafico
fig17.show()

In [ ]:
erros_rl2 = pd.DataFrame({'Métricas avaliadas': ['MAE', 'MAPE', 'WMAPE', 'MSE', 'RMSE'],
                          'Valores obtidos': [round(mean_absolute_error(y_test2, predicao_rl2), 2),
                                              round(mean_absolute_percentage_error(y_test2, predicao_rl2) * 100, 2),
                                              round(mean_absolute_percentage_error(y_test2, predicao_rl2, sample_weight = y_test2) * 100, 2),
                                              round(mean_squared_error(y_test2, predicao_rl2, squared = True), 2),
                                              round(mean_squared_error(y_test2, predicao_rl2, squared = False), 2)]},
                        index = range(0, 5))

erros_rl2

### <b>2.3. Usina 3 </b> <a class = "anchor" id = "chapter_2_3"></a>

In [ ]:
usina3 = df[df['Usina'] == '3'].groupby('Competência').agg({'Compensada':'sum', 'Acumulado':'sum', 
                                                             'Energia_Ativa':'sum'})

usina3

In [ ]:
# Descricao estatistica dos dados

usina3.describe()

In [ ]:
fig18 = go.Figure()

# Configurando o gráfico de barras
fig18.add_trace(go.Bar(x = usina3.index,
                     y = usina3['Compensada'],
                     text = usina3['Compensada'],
                     name = 'Gráfico de barras',
                     textposition = 'inside',
                     textfont_size = 12,
                     marker_color = '#266B6E'))

# Configurando o gráfico de linhas
fig18.add_trace(go.Scatter(x = usina3.index,
                         y = usina3['Compensada'],
                         name = 'Gráfico de linhas',
                         mode = 'lines + markers',
                         marker_color = 'red'))


# Configurando o título do gráfico
fig18.update_layout(title_text = f'Energia Compensada - 3',
                  title_font_size = 30,
                  title_x = 0.5)

# Configurando o título dos eixos
fig18.update_layout(yaxis_title = 'Quantidade de energia',
                  yaxis_title_font_size = 18,
                  xaxis_title = 'Meses',
                  showlegend = False)

# Exibindo as grades de fundo
fig18.update_xaxes(showgrid = True)

# Plotando o gráfico
fig18.show()

### <b>2.3.1. LSTM</b> <a class = "anchor" id = "chapter_2_3_1"></a>

In [ ]:
# Copia do dataset principal

usina3_lstm = usina3.copy(deep = True)

In [ ]:
# Separando quais dados serao utilizados para rodar o modelo

usina3_lstm = usina3_lstm.loc[:,['Compensada']]
usina3_lstm

In [ ]:
# Definindo a frequencia dos dados

usina3_lstm = usina3_lstm.asfreq('MS')

In [ ]:
# Normalizando os dados

scaler_lstm = MinMaxScaler(feature_range = (0, 1))
data_lstm3 = scaler_lstm.fit_transform(usina3_lstm)

In [ ]:
# Dividindo os dados em conjuntos de treino e teste

train_size_lstm3 = int(len(data_lstm3) * 0.7)               #70% para treino e 30% para teste
training_set_lstm3, test_set_lstm3 = data_lstm3[:train_size_lstm3], data_lstm3[train_size_lstm3:]

In [ ]:
# Verificando as dimensoes dos conjuntos

training_set_lstm3.shape, test_set_lstm3.shape

In [ ]:
# Atribuindo valor a janela deslizante

lookback_window = 1

In [ ]:
# Criando a funcao de janela deslizante

def create_dataset(data, lookback_window):
    X, y = [], []

    for i in range(len(data) - lookback_window):
        X.append(data[i:(i + lookback_window), :])
        y.append(data[i + lookback_window, 0])

    return np.array(X), np.array(y)

In [ ]:
# Ajustando os conjuntos de treino e teste

X_train_lstm3, y_train_lstm3 = create_dataset(training_set_lstm3, lookback_window)
X_test_lstm3, y_test_lstm3 = create_dataset(test_set_lstm3, lookback_window)

In [ ]:
# Verificando as dimensoes dos conjuntos

X_test_lstm3.shape, y_test_lstm3.shape

In [ ]:
# Ajustando parametros do modelo LSTM

best_model_lstm = Sequential()
best_model_lstm.add(LSTM(4, input_shape = (lookback_window, 1)))
best_model_lstm.add(Dense(1))
best_model_lstm.compile(loss = 'mean_squared_error', optimizer = 'adam', run_eagerly=True)

history_lstm3 = best_model_lstm.fit(X_train_lstm3,
                                   y_train_lstm3,
                                   epochs = 60,
                                   batch_size = 1,
                                   validation_split = 0.2,
                                   verbose = 2)

In [ ]:
# Rodando modelo para conjunto de teste

y_pred_lstm3 = best_model_lstm.predict(X_test_lstm3)

y_pred_lstm3 = scaler_lstm.inverse_transform(y_pred_lstm3)
y_test_lstm3 = scaler_lstm.inverse_transform(y_test_lstm3.reshape(-1, 1))

In [ ]:
# Criando dataframe com as predicoes

timestamp3 = pd.DataFrame(usina3_lstm.index[-len(X_test_lstm3):])

y_test_df_lstm3 = pd.concat([timestamp3, pd.DataFrame(y_test_lstm3)], axis = 1).rename(columns = {0: 'Compensada'}).dropna()
y_pred_df_lstm3 = pd.concat([timestamp3, pd.DataFrame(y_pred_lstm3)], axis = 1).rename(columns = {0: 'Compensada_predito'}).dropna()


# Ajuste nas datas para corresponderem às datas reais dos registros de teste

y_test_df_lstm3.index = usina3_lstm.index[-len(X_test_lstm3):]
y_pred_df_lstm3.index = usina3_lstm.index[-len(X_test_lstm3):]

In [ ]:
fig19 = go.Figure()

# Configurando o gráfico de linhas
fig19.add_trace(go.Scatter(x = y_test_df_lstm3['Competência'], y = y_test_df_lstm3['Compensada'], mode = 'lines + markers', name = 'Compensada', marker_color = '#266B6E'))
fig19.add_trace(go.Scatter(x = y_pred_df_lstm3['Competência'], y = y_pred_df_lstm3['Compensada_predito'], mode = 'lines + markers', name = 'Predição LSTM', marker_color = '#FF007D'))

# Configurando o título do gráfico
fig19.update_layout(title_text = f'Energia Compensada - 3',
                  title_font_size = 30,
                  title_x = 0.5)

# Configurando o título dos eixos
fig19.update_layout(xaxis_title_font_size = 18,
                  yaxis_title = 'Quantidade de Energia',
                  yaxis_title_font_size = 18,
                  showlegend = True,
                  legend_title = 'Legenda')

# Exibindo as grades de fundo
fig19.update_xaxes(showgrid = True)

# Plotando o gráfico
fig19.show()

In [ ]:
# Obtendo metricas de avaliacao

erros_lstm3 = pd.DataFrame({'Métricas avaliadas': ['MAE', 'MAPE', 'WMAPE', 'MSE', 'RMSE'],
                                'Valores obtidos': [round(mean_absolute_error(y_test_lstm3, y_pred_lstm3), 2),
                                                    round(mean_absolute_percentage_error(y_test_lstm3, y_pred_lstm3) * 100, 2),
                                                    round(mean_absolute_percentage_error(y_test_lstm3, y_pred_lstm3, sample_weight = y_test_lstm3) * 100, 2),
                                                    round(mean_squared_error(y_test_lstm3, y_pred_lstm3, squared = True), 2),
                                                    round(mean_squared_error(y_test_lstm3, y_pred_lstm3, squared = False), 2)]},
                            index = range(0, 5))

erros_lstm3 

### <b>2.3.2. Holt-Winters</b> <a class = "anchor" id = "chapter_2_3_2"></a>

In [ ]:
# Copia do dataset principal

usina3_holtw = usina3.copy(deep = True)

In [ ]:
# Separando quais dados serao utilizados para rodar o modelo

usina3_holtw = usina3_holtw.loc[:,['Compensada']]

In [ ]:
# Definindo a frequencia dos dados

usina3_holtw = usina3_holtw.asfreq('MS')

In [ ]:
# Dividindo dados em conjunto de treino (70%)

training_set_holtwinters3 = usina3_holtw.iloc[:-4, :]
training_set_holtwinters3

In [ ]:
# Dividindo dados em conjunto de teste (30%)

test_set_holtwinters3 = usina3_holtw.iloc[-4:, :]
test_set_holtwinters3

In [ ]:
# Aplicando o modelo

model_holtwinters3 = ExponentialSmoothing(endog = training_set_holtwinters3['Compensada'],     
                                         trend = 'add',                             
                                         seasonal = 'add',
                                         seasonal_periods = 4).fit() 

In [ ]:
predictions_holtwinters3 = pd.Series(model_holtwinters3.forecast(steps = len(test_set_holtwinters3))).rename('HW')
predictions_holtwinters3.index = test_set_holtwinters3.index

In [ ]:
fig20 = go.Figure()

# Configurando o gráfico de linhas
fig20.add_trace(go.Scatter(x = test_set_holtwinters3.index, y = test_set_holtwinters3['Compensada'], mode = 'lines + markers', name = 'Compensada', marker_color = '#266B6E'))
fig20.add_trace(go.Scatter(x = predictions_holtwinters3.index, y = predictions_holtwinters3, mode = 'lines + markers', name = 'Predição Holt-Winters', marker_color = '#FF007D'))

# Configurando o título do gráfico
fig20.update_layout(title_text = f'Energia Compensada - 3',
                  title_font_size = 30,
                  title_x = 0.5)

# Configurando o título dos eixos
fig20.update_layout(xaxis_title_font_size = 18,
                  yaxis_title = 'Quantidade de Energia',
                  yaxis_title_font_size = 18,
                  showlegend = True,
                  legend_title = 'Legenda')

# Exibindo as grades de fundo
fig20.update_xaxes(showgrid = True)

# Plotando o gráfico
fig20.show()

In [ ]:
erros_hw3 = pd.DataFrame({'Métricas avaliadas': ['MAE', 'MAPE', 'WMAPE', 'MSE', 'RMSE'],
                        'Valores obtidos': [round(mean_absolute_error(test_set_holtwinters3['Compensada'], predictions_holtwinters3), 2),
                                            round(mean_absolute_percentage_error(test_set_holtwinters3['Compensada'], predictions_holtwinters3) * 100, 2),
                                            round(mean_absolute_percentage_error(test_set_holtwinters3['Compensada'], predictions_holtwinters3, sample_weight = test_set_holtwinters3['Compensada']) * 100, 2),
                                            round(mean_squared_error(test_set_holtwinters3['Compensada'], predictions_holtwinters3, squared = True), 2),
                                            round(mean_squared_error(test_set_holtwinters3['Compensada'], predictions_holtwinters3, squared = False), 2)]},
                        index = range(0, 5))

erros_hw3

### <b>2.3.3 SARIMAX </b> <a class = "anchor" id = "chapter_2_3_3"></a>

In [ ]:
# Copia do dataset principal

usina3_sarimax = usina3.copy(deep = True)

In [ ]:
# Definindo a frequencia dos dados

usina3_sarimax = usina3_sarimax.asfreq('MS')

In [ ]:
# Definindo o conjunto de treino (70%)

training_set_sarimax3 = usina3_sarimax.iloc[:-4, :]
training_set_sarimax3

In [ ]:
# Definindo o conjunto de teste (30%)

test_set_sarimax3 = usina3_sarimax.iloc[-4:, :]
test_set_sarimax3

In [ ]:
# Definindo as variaveis exogenas de treino

train_exog_sarimax3 = training_set_sarimax3.iloc[:, 1:]
train_exog_sarimax3

In [ ]:
# Definindo as variaveis exogenas de teste

test_exog_sarimax3 = test_set_sarimax3.iloc[:, 1:]
test_exog_sarimax3

In [ ]:
# Aplicando o modelo

model_sarimax3 = auto_arima(y = training_set_sarimax3['Compensada'],
                           X = train_exog_sarimax3,
                           m = 2,                   # Período anual: 12 meses (ajustar de acordo com a frequência escolhida anteriormente)
                           seasonal = True,
                           stepwise = False)

In [ ]:
model_sarimax3.summary()

In [ ]:
predictions_sarimax3 = pd.Series(model_sarimax3.predict(n_periods = 4, X = test_exog_sarimax3)).rename('SARIMAX')
predictions_sarimax3.index = test_set_sarimax3.index

In [ ]:
fig21 = go.Figure()

# Configurando o grafico de linhas
fig21.add_trace(go.Scatter(x = test_set_sarimax3.index, y = test_set_sarimax3['Compensada'], mode = 'lines + markers', name = 'Compensada', marker_color = '#266B6E'))
fig21.add_trace(go.Scatter(x = predictions_sarimax3.index, y = predictions_sarimax3, mode = 'lines + markers', name = 'Predição Sarimax', marker_color = '#FF007D'))


# Configurando o titulo do grafico
fig21.update_layout(title_text = f'Energia Compensada - 3',
                  title_font_size = 30,
                  title_x = 0.5)

# Configurando o titulo dos eixos
fig21.update_layout(xaxis_title_font_size = 18,
                  yaxis_title = 'Quantidade de Energia',
                  yaxis_title_font_size = 18,
                  showlegend = True,
                  legend_title = 'Legenda')

# Exibindo as grades de fundo
fig21.update_xaxes(showgrid = True)

# Plotando o grafico
fig21.show()

In [ ]:
# Calculando os erros

erros_sm3 = pd.DataFrame({'Métricas avaliadas': ['MAE', 'MAPE', 'WMAPE', 'MSE', 'RMSE'],
                        'Valores obtidos': [round(mean_absolute_error(test_set_sarimax3['Compensada'], predictions_sarimax3), 2),
                                            round(mean_absolute_percentage_error(test_set_sarimax3['Compensada'], predictions_sarimax3) * 100, 2),
                                            round(mean_absolute_percentage_error(test_set_sarimax3['Compensada'], predictions_sarimax3, sample_weight = test_set_sarimax3['Compensada']) * 100, 2),
                                            round(mean_squared_error(test_set_sarimax3['Compensada'], predictions_sarimax3, squared = True), 2),
                                            round(mean_squared_error(test_set_sarimax3['Compensada'], predictions_sarimax3, squared = False), 2)]},
                        index = range(0, 5))

erros_sm3

### <b>2.3.4. TensorFlow</b> <a class = "anchor" id = "chapter_2_3_4"></a>

In [ ]:
# Copia do dataset principal

usina3_tensorflow = usina3.copy(deep = True)

In [ ]:
# Definindo a frequencia dos dados

usina3_tensorflow = usina3_tensorflow.asfreq('MS')

In [ ]:
# Definindo o conjunto de treino (70%)

training_set_tensorflow3 = usina3_tensorflow.iloc[:-4, :]
training_set_tensorflow3

In [ ]:
# Definindo os conjuntos de teste (30%)

test_set_tensorflow3 = usina3_tensorflow.iloc[-4:, :]
test_set_tensorflow3

In [ ]:
# Regressores

exog_tensorflow3 = np.asmatrix(usina3_tensorflow.iloc[:, 1:].astype(np.float64))

In [ ]:
# Aplicando o modelo

regressors_tensorflow3 = tfp.sts.LinearRegression(design_matrix = exog_tensorflow3, name = 'regressors')

In [ ]:
# Sazonalidade

Compensada_tensorflow3 = training_set_tensorflow3['Compensada'].astype(np.float64)

In [ ]:
monthly_effect_tensorflow3 = tfp.sts.Seasonal(num_seasons = 3,
                                             num_steps_per_season = np.array([[4]]),
                                             observed_time_series = Compensada_tensorflow3,
                                             name = 'month_effect')

In [ ]:
# Definindo a variavel autoregressive

autoregressive_tensorflow3 = tfp.sts.Autoregressive(order = 1, observed_time_series = Compensada_tensorflow3, name = 'autoregressive')

In [ ]:
# Definindo a variavel trend

trend_tensorflow3 = tfp.sts.LocalLinearTrend(observed_time_series = Compensada_tensorflow3, name = 'trend')

In [ ]:
# Aplicando o modelo novamente

model_tensorflow3 = tfp.sts.Sum([regressors_tensorflow3,
                                monthly_effect_tensorflow3,
                                autoregressive_tensorflow3,
                                trend_tensorflow3],
                                observed_time_series = Compensada_tensorflow3)

In [ ]:
samples_tensorflow3, kernel_results_tensorflow3 = tfp.sts.fit_with_hmc(model = model_tensorflow3,
                                                                     observed_time_series = Compensada_tensorflow3,
                                                                     num_results = 100,
                                                                     num_warmup_steps = 50,
                                                                     num_leapfrog_steps = 15,
                                                                     num_variational_steps = 150,
                                                                     seed = 1502)

In [ ]:
forecast_tensorflow3 = tfp.sts.forecast(model = model_tensorflow3,
                                       observed_time_series = Compensada_tensorflow3,
                                       parameter_samples = samples_tensorflow3,
                                       num_steps_forecast = len(test_set_tensorflow3))

In [ ]:
predictions_tensorflow3 = pd.Series(forecast_tensorflow3.mean()[:, 0], name = 'TFP')
predictions_tensorflow3.index = test_set_tensorflow3.index

In [ ]:
fig22 = go.Figure()

# Configurando o gráfico de linhas
fig22.add_trace(go.Scatter(x = test_set_tensorflow3.index, y = test_set_tensorflow3['Compensada'], mode = 'lines + markers', name = 'Compensada', marker_color = '#266B6E'))
fig22.add_trace(go.Scatter(x = predictions_tensorflow3.index, y = predictions_tensorflow3, mode = 'lines + markers', name = 'Predição Tensorflow', marker_color = '#FF007D'))

# Configurando o título do gráfico
fig22.update_layout(title_text = f'Energia Compensada - 3',
                  title_font_size = 30,
                  title_x = 0.5)

# Configurando o título dos eixos
fig22.update_layout(xaxis_title_font_size = 18,
                  yaxis_title = 'Quantidade de Energia',
                  yaxis_title_font_size = 18,
                  showlegend = True,
                  legend_title = 'Legenda')

# Exibindo as grades de fundo
fig22.update_xaxes(showgrid = True)

# Plotando o gráfico
fig22.show()

In [ ]:
erros_tf3 = pd.DataFrame({'Métricas avaliadas': ['MAE', 'MAPE', 'WMAPE', 'MSE', 'RMSE'],
                                 'Valores obtidos': [round(mean_absolute_error(test_set_tensorflow3['Compensada'], predictions_tensorflow3), 2),
                                                     round(mean_absolute_percentage_error(test_set_tensorflow3['Compensada'], predictions_tensorflow3) * 100, 2),
                                                     round(mean_absolute_percentage_error(test_set_tensorflow3['Compensada'], predictions_tensorflow3, sample_weight = test_set_tensorflow3['Compensada']) * 100, 2),
                                                     round(mean_squared_error(test_set_tensorflow3['Compensada'], predictions_tensorflow3, squared = True), 2),
                                                     round(mean_squared_error(test_set_tensorflow3['Compensada'], predictions_tensorflow3, squared = False), 2)]},
                                index = range(0, 5))

erros_tf3

### <b>Organizando os dados para os modelos seguintes</b>

In [ ]:
usina3_modelos = usina3.copy(deep = True)
usina3_modelos

In [ ]:
# Dividindo os dados em conjuntos de treino e teste a partir de um corte temporal

df_train3 = usina3_modelos.loc[usina3_modelos.index < '2023-03-01']
df_test3 = usina3_modelos.loc[usina3_modelos.index >= '2023-03-01']

In [ ]:
# Separando os conjuntos X e Y de acordo com as colunas de dados

X_train3 = df_train3[['Acumulado', 'Energia_Ativa']]

y_train3 = df_train3['Compensada']
y_train3 = np.array(y_train3).reshape(-1, 1)

X_test3 = df_test3[['Acumulado', 'Energia_Ativa']]

y_test3 = df_test3['Compensada']
y_test3 = np.array(y_test3).reshape(-1, 1)

### <b>2.3.5 XGBoost </b> <a class = "anchor" id = "chapter_2_3_5"></a>

In [ ]:
# Instanciando o modelo
modelo_xgb = xgb.XGBRegressor(random_state = 0, max_depth=2)

# Ajustando o modelo
modelo_xgb3 = modelo_xgb.fit(X_train3, y_train3)

# Fazendo as predicoes
predicao_xgb3 = modelo_xgb3.predict(X_test3)

print('XGBoost')
# r2 dos dados de treino
print('R² do treino = {}'.format(modelo_xgb3.score(X_train3, y_train3)))

# r2 dos dados de teste
print('R² do teste = %s' % metrics.r2_score(y_test3, predicao_xgb3))

# Erro medio quadratico dos dados de teste e preditos
print('Raiz quadrada do erro médio quadrático dos resultados preditos = {}'.format(np.sqrt(mean_squared_error(y_test3, predicao_xgb3))))

# Erro medio absoluto dos dados de teste e treino
print('Erro médio absoluto dos resultados preditos = {}'.format(mean_absolute_error(y_test3, predicao_xgb3).round(2)))

In [ ]:
#Verificando quais variaveis impactam mais o modelo

fi3 = pd.DataFrame(data=modelo_xgb3.feature_importances_,
             index=modelo_xgb3.feature_names_in_,
             columns=['importância'])
fi3.sort_values('importância').plot(kind='barh', title='Importância das Variáveis - 3')
plt.show()

In [ ]:
# Fazendo um gridsearch para encontrar os melhores parametros

# Instanciando o modelo
modelo_xgb = xgb.XGBRegressor(random_state = 0)

# Definindo o espaço de busca dos hiperparametros
param_grid = {'learning_rate': np.linspace(0.001, 0.1, num = 20), 'n_estimators': np.arange(100, 200, 10),
               'max_depth': [2, 3, 4, 5]}

# Chamando o griddsearch e fazendo a busca aleatoria de acordo com os hiperparametros fornecidos acima
grid_search3 =  GridSearchCV(modelo_xgb, param_grid, scoring='neg_mean_squared_error', cv = 5)
grid_search3.fit(X_train3, y_train3)

# Acessando os melhores hiperparametros encontrados
melhores_parametros3 = grid_search3.best_params_
melhores_parametros3

In [ ]:
#Rodando o modelo novamente, agora com os novos parametros encontrados

# Instanciando o modelo
modelo_xgb = xgb.XGBRegressor(random_state = 0, n_estimators = 100, max_depth = 3, learning_rate = 0.058)

# Ajustando o modelo
modelo_xgb3 = modelo_xgb.fit(X_train3, y_train3)

# Fazendo as predicoes
predicao_xgb3 = modelo_xgb3.predict(X_test3)

print('XGBoost')
# r2 dos dados de treino
print('R² do treino = {}'.format(modelo_xgb3.score(X_train3, y_train3).round(2)))

# r2 dos dados de teste
print('R² do teste = %s' % metrics.r2_score(y_test3, predicao_xgb3).round(2))

# Erro quadratico medio dos dados de teste e preditos
print('Raiz quadrada do erro quadratico medio dos resultados preditos = {}'.format(np.sqrt(mean_squared_error(y_test3, predicao_xgb3)).round(2)))

# Erro medio absoluto dos dados de teste e treino
print('Erro medio absoluto dos resultados preditos = {}'.format(mean_absolute_error(y_test3, predicao_xgb3).round(2)))

In [ ]:
predictions_xgb3 = pd.DataFrame(y_test3, columns=['Compensada'])
predictions_xgb3.index = X_test1.index

predicao_xgb3 = pd.DataFrame(predicao_xgb3)
predicao_xgb3.index = X_test3.index

predictions_xgb3['Predição XGB'] = predicao_xgb3

In [ ]:
fig23 = go.Figure()

# Configurando o grafico de linhas
fig23.add_trace(go.Scatter(x = predictions_xgb3.index, y = predictions_xgb3['Compensada'], mode = 'lines + markers', name = 'Compensada', marker_color = '#266B6E'))
fig23.add_trace(go.Scatter(x = predictions_xgb3.index, y = predictions_xgb3['Predição XGB'], mode = 'lines + markers', name = 'Predição XGBoost', marker_color = '#FF007D'))

# Configurando o titulo do grafico
fig23.update_layout(title_text = f'Energia Compensada - 3',
                  title_font_size = 30,
                  title_x = 0.5)

# Configurando o titulo dos eixos
fig23.update_layout(xaxis_title_font_size = 18,
                  yaxis_title = 'Quantidade de Energia',
                  yaxis_title_font_size = 18,
                  showlegend = True,
                  legend_title = 'Legenda')

# Exibindo as grades de fundo
fig23.update_xaxes(showgrid = True)

# Plotando o grafico
fig23.show()

In [ ]:
erros_xgb3 = pd.DataFrame({'Métricas avaliadas': ['MAE', 'MAPE', 'WMAPE', 'MSE', 'RMSE'],
                          'Valores obtidos': [round(mean_absolute_error(y_test3, predicao_xgb3), 2),
                                              round(mean_absolute_percentage_error(y_test3, predicao_xgb3) * 100, 2),
                                              round(mean_absolute_percentage_error(y_test3, predicao_xgb3, sample_weight = y_test3) * 100, 2),
                                              round(mean_squared_error(y_test3, predicao_xgb3, squared = True), 2),
                                              round(mean_squared_error(y_test3, predicao_xgb3, squared = False), 2)]},
                        index = range(0, 5))

erros_xgb3

### <b>2.3.6 MLP </b> <a class = "anchor" id = "chapter_2_3_6"></a>

In [ ]:
# Instanciando o modelo
nnet = MLPRegressor(random_state = 0)

# Ajustando o modelo
modelo_nnet3 = nnet.fit(X_train3, y_train3)

# Fazendo as predicoes
predicao_nnet3 = modelo_nnet3.predict(X_test3)

print('MLP')
# r2 dos dados de treino
print('R² do treino = {}'.format(modelo_nnet3.score(X_train3, y_train3).round(2)))

# r2 dos dados de teste
print('R² do teste = %s' % metrics.r2_score(y_test3, predicao_nnet3).round(2))

In [ ]:
predictions_nnet3 = pd.DataFrame(y_test3, columns=['Compensada'])
predictions_nnet3.index = X_test3.index

predicao_nnet3 = pd.DataFrame(predicao_nnet3)
predicao_nnet3.index = X_test3.index

predictions_nnet3['Predição MLP'] = predicao_nnet3

In [ ]:
fig24 = go.Figure()

# Configurando o grafico de linhas
fig24.add_trace(go.Scatter(x = predictions_nnet3.index, y = predictions_nnet3['Compensada'], mode = 'lines + markers', name = 'Compensada', marker_color = '#266B6E'))
fig24.add_trace(go.Scatter(x = predictions_nnet3.index, y = predictions_nnet3['Predição MLP'], mode = 'lines + markers', name = 'Predição MLP', marker_color = '#FF007D'))

# Configurando o titulo do grafico
fig24.update_layout(title_text = f'Energia Compensada - 3',
                  title_font_size = 30,
                  title_x = 0.5)

# Configurando o titulo dos eixos
fig24.update_layout(xaxis_title_font_size = 18,
                  yaxis_title = 'Quantidade de Energia',
                  yaxis_title_font_size = 18,
                  showlegend = True,
                  legend_title = 'Legenda')

# Exibindo as grades de fundo
fig24.update_xaxes(showgrid = True)

# Plotando o grafico
fig24.show()

In [ ]:
erros_mlp3 = pd.DataFrame({'Métricas avaliadas': ['MAE', 'MAPE', 'WMAPE', 'MSE', 'RMSE'],
                          'Valores obtidos': [round(mean_absolute_error(y_test3, predicao_nnet3), 2),
                                              round(mean_absolute_percentage_error(y_test3, predicao_nnet3) * 100, 2),
                                              round(mean_absolute_percentage_error(y_test3, predicao_nnet3, sample_weight = y_test3) * 100, 2),
                                              round(mean_squared_error(y_test3, predicao_nnet3, squared = True), 2),
                                              round(mean_squared_error(y_test3, predicao_nnet3, squared = False), 2)]},
                        index = range(0, 5))

erros_mlp3


### <b>2.3.7 Random Forest </b> <a class = "anchor" id = "chapter_2_3_7"></a>

In [ ]:
# Instanciando o modelo
rf = RandomForestRegressor(random_state = 0)

# Ajustando o modelo
model_rf3 = rf.fit(X_train3, y_train3)

# Fazendo as predicoes
predicao_rf3 = model_rf3.predict(X_test3)

print('\n')

print('RANDOMFOREST')
# r2 dos dados de treino
print('R² do treino = {}'.format(model_rf3.score(X_train3, y_train3).round(2)))

# r2 dos dados de teste
print('R² do teste = %s' % metrics.r2_score(y_test3, predicao_rf3).round(2))

In [ ]:
predictions_rf3 = pd.DataFrame(y_test3, columns=['Compensada'])
predictions_rf3.index = X_test3.index

predicao_rf3 = pd.DataFrame(predicao_rf3)
predicao_rf3.index = X_test3.index

predictions_rf3['Predição Random Forest'] = predicao_rf3

In [ ]:
fig25 = go.Figure()

# Configurando o grafico de linhas
fig25.add_trace(go.Scatter(x = predictions_rf3.index, y = predictions_rf3['Compensada'], mode = 'lines + markers', name = 'Compensada', marker_color = '#266B6E'))
fig25.add_trace(go.Scatter(x = predictions_rf3.index, y = predictions_rf3['Predição Random Forest'], mode = 'lines + markers', name = 'Predição Random Forest', marker_color = '#FF007D'))

# Configurando o titulo do grafico
fig25.update_layout(title_text = f'Energia Compensada - 3',
                  title_font_size = 30
                  title_x = 0.5)

# Configurando o titulo dos eixos
fig25.update_layout(xaxis_title_font_size = 18,
                  yaxis_title = 'Quantidade de Energia',
                  yaxis_title_font_size = 18,
                  showlegend = True,
                  legend_title = 'Legenda')

# Exibindo as grades de fundo
fig25.update_xaxes(showgrid = True)

# Plotando o grafico
fig25.show()

In [ ]:
erros_rf3 = pd.DataFrame({'Métricas avaliadas': ['MAE', 'MAPE', 'WMAPE', 'MSE', 'RMSE'],
                          'Valores obtidos': [round(mean_absolute_error(y_test3, predicao_rf3), 2),
                                              round(mean_absolute_percentage_error(y_test3, predicao_rf3) * 100, 2),
                                              round(mean_absolute_percentage_error(y_test3, predicao_rf3, sample_weight = y_test3) * 100, 2),
                                              round(mean_squared_error(y_test3, predicao_rf3, squared = True), 2),
                                              round(mean_squared_error(y_test3, predicao_rf3, squared = False), 2)]},
                        index = range(0, 5))

erros_rf3

### <b>2.3.8 Regressão Linear </b> <a class = "anchor" id = "chapter_2_3_8"></a>

In [ ]:
# Instanciando o modelo 
rl = LinearRegression()

# Ajustando o modelo
modelo_rl3 = rl.fit(X_train3, y_train3)

# Fazendo as predicoes
predicao_rl3 = modelo_rl3.predict(X_test3)

print('REGRESSÃO LINEAR')
# r2 dos dados de treino
print('R² do treino = {}'.format(modelo_rl3.score(X_train3, y_train3).round(2)))

# r2 dos dados de teste
print('R² do teste = %s' % metrics.r2_score(y_test3, predicao_rl3).round(2))

In [ ]:
predictions_rl3 = pd.DataFrame(y_test3, columns=['Compensada'])
predictions_rl3.index = X_test3.index

predicao_rl3 = pd.DataFrame(predicao_rl3)
predicao_rl3.index = X_test3.index

predictions_rl3['Predição Regressão Linear'] = predicao_rl3

In [ ]:
fig26 = go.Figure()

# Configurando o grafico de linhas
fig26.add_trace(go.Scatter(x = predictions_rl3.index, y = predictions_rl3['Compensada'], mode = 'lines + markers', name = 'Compensada', marker_color = '#266B6E'))
fig26.add_trace(go.Scatter(x = predictions_rl3.index, y = predictions_rl3['Predição Regressão Linear'], mode = 'lines + markers', name = 'Predição Regressão Linear', marker_color = '#FF007D'))

# Configurando o titulo do grafico
fig26.update_layout(title_text = f'Energia Compensada - 3',
                  title_font_size = 30,
                  title_x = 0.5)

# Configurando o titulo dos eixos
fig26.update_layout(xaxis_title_font_size = 18,
                  yaxis_title = 'Quantidade de Energia',
                  yaxis_title_font_size = 18,
                  showlegend = True,
                  legend_title = 'Legenda')

# Exibindo as grades de fundo
fig26.update_xaxes(showgrid = True)

# Plotando o grafico
fig26.show()

In [ ]:
erros_rl3 = pd.DataFrame({'Métricas avaliadas': ['MAE', 'MAPE', 'WMAPE', 'MSE', 'RMSE'],
                          'Valores obtidos': [round(mean_absolute_error(y_test3, predicao_rl3), 2),
                                              round(mean_absolute_percentage_error(y_test3, predicao_rl3) * 100, 2),
                                              round(mean_absolute_percentage_error(y_test3, predicao_rl3, sample_weight = y_test3) * 100, 2),
                                              round(mean_squared_error(y_test3, predicao_rl3, squared = True), 2),
                                              round(mean_squared_error(y_test3, predicao_rl3, squared = False), 2)]},
                        index = range(0, 5))


erros_rl3

### <b>2.4. Usina 4 </b> <a class = "anchor" id = "chapter_2_4"></a>

In [ ]:
usina4 = df[df['Usina'] == '4'].groupby('Competência').agg({'Compensada':'sum', 'Acumulado':'sum', 
                                                             'Energia_Ativa':'sum'})

usina4

In [ ]:
# Descricao estatistica dos dados

usina4.describe()

In [ ]:
fig27 = go.Figure()

# Configurando o grafico de barras
fig27.add_trace(go.Bar(x = usina4.index,
                     y = usina4['Compensada'],
                     text = usina4['Compensada'],
                     name = 'Gráfico de barras',
                     textposition = 'inside',
                     textfont_size = 12,
                     marker_color = '#266B6E'))

# Configurando o grafico de linhas
fig27.add_trace(go.Scatter(x = usina4.index,
                         y = usina4['Compensada'],
                         name = 'Gráfico de linhas',
                         mode = 'lines + markers',
                         marker_color = 'red'))


# Configurando o titulo do grafico
fig27.update_layout(title_text = f'Energia Compensada - 4',
                  title_font_size = 30,
                  title_x = 0.5)

# Configurando o titulo dos eixos
fig27.update_layout(yaxis_title = 'Quantidade de energia',
                  yaxis_title_font_size = 18,
                  xaxis_title = 'Meses',
                  showlegend = False)

# Exibindo as grades de fundo
fig27.update_xaxes(showgrid = True)

# Plotando o grafico
fig27.show()

### <b>2.4.1. LSTM</b> <a class = "anchor" id = "chapter_2_4_1"></a>

In [ ]:
# Copia do dataset principal

usina4_lstm = usina4.copy(deep = True)

In [ ]:
# Separando quais dados serao utilizados para rodar o modelo

usina4_lstm = usina4_lstm.loc[:,['Compensada']]

In [ ]:
# Definindo a frequencia dos dados

usina4_lstm = usina4_lstm.asfreq('MS')

In [ ]:
# Normalizando os dados

scaler_lstm = MinMaxScaler(feature_range = (0, 1))
data_lstm4 = scaler_lstm.fit_transform(usina4_lstm)

In [ ]:
# Dividindo os dados em conjuntos de treino e teste

train_size_lstm4 = int(len(data_lstm4) * 0.7)               #70% para treino e 30% para teste
training_set_lstm4, test_set_lstm4 = data_lstm4[:train_size_lstm4], data_lstm4[train_size_lstm4:]

In [ ]:
# Verificando as dimensoes dos conjuntos

training_set_lstm4.shape, test_set_lstm4.shape

In [ ]:
# Atribuindo valor a janela deslizante

lookback_window = 1

In [ ]:
# Criando a funcao de janela deslizante

def create_dataset(data, lookback_window):
    X, y = [], []

    for i in range(len(data) - lookback_window):
        X.append(data[i:(i + lookback_window), :])
        y.append(data[i + lookback_window, 0])

    return np.array(X), np.array(y)

In [ ]:
# Ajustando os conjuntos de treino e teste

X_train_lstm4, y_train_lstm4 = create_dataset(training_set_lstm4, lookback_window)
X_test_lstm4, y_test_lstm4 = create_dataset(test_set_lstm4, lookback_window)

In [ ]:
# Verificando as dimensoes dos conjuntos

X_test_lstm4.shape, y_test_lstm4.shape

In [ ]:
# Ajustando parametros do modelo LSTM

best_model_lstm = Sequential()
best_model_lstm.add(LSTM(4, input_shape = (lookback_window, 1)))
best_model_lstm.add(Dense(1))
best_model_lstm.compile(loss = 'mean_squared_error', optimizer = 'adam', run_eagerly=True)

history_lstm4 = best_model_lstm.fit(X_train_lstm4,
                                   y_train_lstm4,
                                   epochs = 70,
                                   batch_size = 1,
                                   validation_split = 0.2,
                                   verbose = 2)

In [ ]:
# Rodando modelo para conjunto de teste

y_pred_lstm4 = best_model_lstm.predict(X_test_lstm4)

y_pred_lstm4 = scaler_lstm.inverse_transform(y_pred_lstm4)
y_test_lstm4 = scaler_lstm.inverse_transform(y_test_lstm4.reshape(-1, 1))

In [ ]:
# Criando dataframe com as predicoes

timestamp4 = pd.DataFrame(usina4_lstm.index[-len(X_test_lstm4):])

y_test_df_lstm4 = pd.concat([timestamp4, pd.DataFrame(y_test_lstm4)], axis = 1).rename(columns = {0: 'Compensada'}).dropna()
y_pred_df_lstm4 = pd.concat([timestamp4, pd.DataFrame(y_pred_lstm4)], axis = 1).rename(columns = {0: 'Compensada_predito'}).dropna()


# Ajuste nas datas para corresponderem as datas reais dos registros de teste

y_test_df_lstm4.index = usina4_lstm.index[-len(X_test_lstm4):]
y_pred_df_lstm4.index = usina4_lstm.index[-len(X_test_lstm4):]

In [ ]:
fig28 = go.Figure()

# Configurando o grafico de linhas
fig28.add_trace(go.Scatter(x = y_test_df_lstm4['Competência'], y = y_test_df_lstm4['Compensada'], mode = 'lines + markers', name = 'Compensada', marker_color = '#266B6E'))
fig28.add_trace(go.Scatter(x = y_pred_df_lstm4['Competência'], y = y_pred_df_lstm4['Compensada_predito'], mode = 'lines + markers', name = 'Predição LSTM', marker_color = '#FF007D'))

# Configurando o titulo do grafico
fig28.update_layout(title_text = f'Energia Compensada - 4',
                  title_font_size = 30,
                  title_x = 0.5)

# Configurando o titulo dos eixos
fig28.update_layout(xaxis_title_font_size = 18,
                  yaxis_title = 'Quantidade de Energia',
                  yaxis_title_font_size = 18,
                  showlegend = True,
                  legend_title = 'Legenda')

# Exibindo as grades de fundo
fig28.update_xaxes(showgrid = True)

# Plotando o grafico
fig28.show()

In [ ]:
# Obtendo metricas de avaliacao

erros_lstm4 = pd.DataFrame({'Métricas avaliadas': ['MAE', 'MAPE', 'WMAPE', 'MSE', 'RMSE'],
                                'Valores obtidos': [round(mean_absolute_error(y_test_lstm4, y_pred_lstm4), 2),
                                                    round(mean_absolute_percentage_error(y_test_lstm4, y_pred_lstm4) * 100, 2),
                                                    round(mean_absolute_percentage_error(y_test_lstm4, y_pred_lstm4, sample_weight = y_test_lstm4) * 100, 2),
                                                    round(mean_squared_error(y_test_lstm4, y_pred_lstm4, squared = True), 2),
                                                    round(mean_squared_error(y_test_lstm4, y_pred_lstm4, squared = False), 2)]},
                            index = range(0, 5))

erros_lstm4

### <b>2.4.2. Holt-Winters </b> <a class = "anchor" id = "chapter_2_4_2"></a>

In [ ]:
# Copia do dataset principal

usina4_holtw = usina4.copy(deep = True)

In [ ]:
# Separando quais dados serao utilizados para rodar o modelo

usina4_holtw = usina4_holtw.loc[:,['Compensada']]

In [ ]:
# Definindo a frequencia dos dados

usina4_holtw = usina4_holtw.asfreq('MS')

In [ ]:
# Dividindo dados em conjunto de treino (70%)

training_set_holtwinters4 = usina4_holtw.iloc[:-4, :]
training_set_holtwinters4

In [ ]:
# Dividindo dados em conjunto de teste (30%)

test_set_holtwinters4 = usina4_holtw.iloc[-4:, :]
test_set_holtwinters4

In [ ]:
# Aplicando o modelo

model_holtwinters4 = ExponentialSmoothing(endog = training_set_holtwinters4['Compensada'],     
                                         trend = 'add',                             
                                         seasonal = 'add',
                                         seasonal_periods = 3).fit()        

In [ ]:
predictions_holtwinters4 = pd.Series(model_holtwinters4.forecast(steps = len(test_set_holtwinters4))).rename('HW')
predictions_holtwinters4.index = test_set_holtwinters4.index

In [ ]:
fig29 = go.Figure()

# Configurando o grafico de linhas
fig29.add_trace(go.Scatter(x = test_set_holtwinters4.index, y = test_set_holtwinters4['Compensada'], mode = 'lines + markers', name = 'Compensada', marker_color = '#266B6E'))
fig29.add_trace(go.Scatter(x = predictions_holtwinters4.index, y = predictions_holtwinters4, mode = 'lines + markers', name = 'Predição Holt-Winters', marker_color = '#FF007D'))

# Configurando o titulo do grafico
fig29.update_layout(title_text = f'Energia Compensada - 4',
                  title_font_size = 30,
                  title_x = 0.5)

# Configurando o titulo dos eixos
fig29.update_layout(xaxis_title_font_size = 18,
                  yaxis_title = 'Quantidade de Energia',
                  yaxis_title_font_size = 18,
                  showlegend = True,
                  legend_title = 'Legenda')

# Exibindo as grades de fundo
fig29.update_xaxes(showgrid = True)

# Plotando o grafico
fig29.show()

In [ ]:
# Obtendo metricas de avaliacao

erros_hw4 = pd.DataFrame({'Métricas avaliadas': ['MAE', 'MAPE', 'WMAPE', 'MSE', 'RMSE'],
                        'Valores obtidos': [round(mean_absolute_error(test_set_holtwinters4['Compensada'], predictions_holtwinters4), 2),
                                            round(mean_absolute_percentage_error(test_set_holtwinters4['Compensada'], predictions_holtwinters4) * 100, 2),
                                            round(mean_absolute_percentage_error(test_set_holtwinters4['Compensada'], predictions_holtwinters4, sample_weight = test_set_holtwinters4['Compensada']) * 100, 2),
                                            round(mean_squared_error(test_set_holtwinters4['Compensada'], predictions_holtwinters4, squared = True), 2),
                                            round(mean_squared_error(test_set_holtwinters4['Compensada'], predictions_holtwinters4, squared = False), 2)]},
                        index = range(0, 5))

erros_hw4

### <b>2.4.3 SARIMAX </b> <a class = "anchor" id = "chapter_2_4_3"></a>

In [ ]:
# Copia do dataset principal

usina4_sarimax = usina4.copy(deep = True)

In [ ]:
# Definindo a frequencia dos dados

usina4_sarimax = usina4_sarimax.asfreq('MS')

In [ ]:
# Definindo os conjuntos de treino e teste

training_set_sarimax4 = usina4_sarimax.iloc[:-4, :]
training_set_sarimax4

In [ ]:
test_set_sarimax4 = usina4_sarimax.iloc[-4:, :]
test_set_sarimax4

In [ ]:
# Definindo as variaveis exogenas

train_exog_sarimax4 = training_set_sarimax4.iloc[:, 1:]
train_exog_sarimax4

In [ ]:
test_exog_sarimax4 = test_set_sarimax4.iloc[:, 1:]
test_exog_sarimax4

In [ ]:
# Aplicando o modelo

model_sarimax4 = auto_arima(y = training_set_sarimax4['Compensada'],
                           X = train_exog_sarimax4,
                           m = 2,                   # Período anual: 12 meses (ajustar de acordo com a frequência escolhida anteriormente)
                           seasonal = True,
                           stepwise = False)

In [ ]:
model_sarimax4.summary()

In [ ]:
predictions_sarimax4 = pd.Series(model_sarimax4.predict(n_periods = 4, X = test_exog_sarimax4)).rename('SARIMAX')
predictions_sarimax4.index = test_set_sarimax4.index

In [ ]:
fig30 = go.Figure()

# Configurando o grafico de linhas
fig30.add_trace(go.Scatter(x = test_set_sarimax4.index, y = test_set_sarimax4['Compensada'], mode = 'lines + markers', name = 'Compensada', marker_color = '#266B6E'))
fig30.add_trace(go.Scatter(x = predictions_sarimax4.index, y = predictions_sarimax4, mode = 'lines + markers', name = 'Predição Sarimax', marker_color = '#FF007D'))


# Configurando o titulo do grafico
fig30.update_layout(title_text = f'Energia Compensada - 4',
                  title_font_size = 30,
                  title_x = 0.5)

# Configurando o titulo dos eixos
fig30.update_layout(xaxis_title_font_size = 18,
                  yaxis_title = 'Quantidade de Energia',
                  yaxis_title_font_size = 18,
                  showlegend = True,
                  legend_title = 'Legenda')

# Exibindo as grades de fundo
fig30.update_xaxes(showgrid = True)

# Plotando o grafico
fig30.show()

In [ ]:
# Obtendo metricas de avaliacao

erros_sm4 = pd.DataFrame({'Métricas avaliadas': ['MAE', 'MAPE', 'WMAPE', 'MSE', 'RMSE'],
                        'Valores obtidos': [round(mean_absolute_error(test_set_sarimax4['Compensada'], predictions_sarimax4), 2),
                                            round(mean_absolute_percentage_error(test_set_sarimax4['Compensada'], predictions_sarimax4) * 100, 2),
                                            round(mean_absolute_percentage_error(test_set_sarimax4['Compensada'], predictions_sarimax4, sample_weight = test_set_sarimax4['Compensada']) * 100, 2),
                                            round(mean_squared_error(test_set_sarimax4['Compensada'], predictions_sarimax4, squared = True), 2),
                                            round(mean_squared_error(test_set_sarimax4['Compensada'], predictions_sarimax4, squared = False), 2)]},
                        index = range(0, 5))

erros_sm4

### <b>2.4.4. TensorFlow</b> <a class = "anchor" id = "chapter_2_4_4"></a>

In [ ]:
# Copia do dataset principal

usina4_tensorflow = usina4.copy(deep = True)

In [ ]:
# Definindo a frequencia dos dados

usina4_tensorflow = usina4_tensorflow.asfreq('MS')
usina4_tensorflow

In [ ]:
# Definindo o conjunto de treino (70%)

training_set_tensorflow4 = usina4_tensorflow.iloc[:-4, :]
training_set_tensorflow4

In [ ]:
# Definindo o conjunto de teste (30%)

test_set_tensorflow4 = usina4_tensorflow.iloc[-4:, :]
test_set_tensorflow4

In [ ]:
# Regressores

exog_tensorflow4 = np.asmatrix(usina4_tensorflow.iloc[:, 1:].astype(np.float64))

In [ ]:
# Aplicando o modelo

regressors_tensorflow4 = tfp.sts.LinearRegression(design_matrix = exog_tensorflow4, name = 'regressors')

In [ ]:
# Sazonalidade

Compensada_tensorflow4 = training_set_tensorflow4['Compensada'].astype(np.float64)

In [ ]:
monthly_effect_tensorflow4 = tfp.sts.Seasonal(num_seasons = 3,
                                             num_steps_per_season = np.array([[4]]),
                                             observed_time_series = Compensada_tensorflow4,
                                             name = 'month_effect')

In [ ]:
# Definindo a variavel autoregressive

autoregressive_tensorflow4 = tfp.sts.Autoregressive(order = 1, observed_time_series = Compensada_tensorflow4, name = 'autoregressive')

In [ ]:
# Definindo a variavel trend

trend_tensorflow4 = tfp.sts.LocalLinearTrend(observed_time_series = Compensada_tensorflow4, name = 'trend')

In [ ]:
# Aplicando o modelo novamente

model_tensorflow4 = tfp.sts.Sum([regressors_tensorflow4,
                                monthly_effect_tensorflow4,
                                autoregressive_tensorflow4,
                                trend_tensorflow4],
                                observed_time_series = Compensada_tensorflow4)

In [ ]:
samples_tensorflow4, kernel_results_tensorflow4 = tfp.sts.fit_with_hmc(model = model_tensorflow4,
                                                                     observed_time_series = Compensada_tensorflow4,
                                                                     num_results = 100,
                                                                     num_warmup_steps = 50,
                                                                     num_leapfrog_steps = 15,
                                                                     num_variational_steps = 150,
                                                                     seed = 1502)

In [ ]:
forecast_tensorflow4 = tfp.sts.forecast(model = model_tensorflow4,
                                       observed_time_series = Compensada_tensorflow4,
                                       parameter_samples = samples_tensorflow4,
                                       num_steps_forecast = len(test_set_tensorflow4))

In [ ]:
predictions_tensorflow4 = pd.Series(forecast_tensorflow4.mean()[:, 0], name = 'TFP')
predictions_tensorflow4.index = test_set_tensorflow4.index

In [ ]:
fig31 = go.Figure()

# Configurando o grafico de linhas
fig31.add_trace(go.Scatter(x = test_set_tensorflow4.index, y = test_set_tensorflow4['Compensada'], mode = 'lines + markers', name = 'Compensada', marker_color = '#266B6E'))
fig31.add_trace(go.Scatter(x = predictions_tensorflow4.index, y = predictions_tensorflow4, mode = 'lines + markers', name = 'Predição Tensorflow', marker_color = '#FF007D'))

# Configurando o titulo do grafico
fig31.update_layout(title_text = f'Energia Compensada - 4',
                  title_font_size = 30,
                  title_x = 0.5)

# Configurando o titulo dos eixos
fig31.update_layout(xaxis_title_font_size = 18,
                  yaxis_title = 'Quantidade de Energia',
                  yaxis_title_font_size = 18,
                  showlegend = True,
                  legend_title = 'Legenda')

# Exibindo as grades de fundo
fig31.update_xaxes(showgrid = True)

# Plotando o grafico
fig31.show()

In [ ]:
# Comparando valores reais e preditos

pd.DataFrame({'Data': test_set_tensorflow4.index,
              'Valor real': test_set_tensorflow4['Compensada'],
              'Valor predito': round(predictions_tensorflow4, 2),
              '| Diferença |': round(np.abs(test_set_tensorflow4['Compensada'] - predictions_tensorflow4), 2)}).reset_index(drop = True)

In [ ]:
erros_tf4 = pd.DataFrame({'Métricas avaliadas': ['MAE', 'MAPE', 'WMAPE', 'MSE', 'RMSE'],
                                 'Valores obtidos': [round(mean_absolute_error(test_set_tensorflow4['Compensada'], predictions_tensorflow4), 2),
                                                     round(mean_absolute_percentage_error(test_set_tensorflow4['Compensada'], predictions_tensorflow4) * 100, 2),
                                                     round(mean_absolute_percentage_error(test_set_tensorflow4['Compensada'], predictions_tensorflow4, sample_weight = test_set_tensorflow4['Compensada']) * 100, 2),
                                                     round(mean_squared_error(test_set_tensorflow4['Compensada'], predictions_tensorflow4, squared = True), 2),
                                                     round(mean_squared_error(test_set_tensorflow4['Compensada'], predictions_tensorflow4, squared = False), 2)]},
                                index = range(0, 5))

erros_tf4

### <b>Organizando os dados para os modelos seguintes</b>

In [ ]:
usina4_modelos = usina4.copy(deep = True)

In [ ]:
# Dividindo os dados em conjuntos de treino e teste a partir de um corte temporal

df_train4 = usina4_modelos.loc[usina4_modelos.index < '2023-03-01']
df_test4 = usina4_modelos.loc[usina4_modelos.index >= '2023-03-01']

In [ ]:
# Separando os conjuntos X e Y de acordo com as colunas de dados

X_train4 = df_train4[['Acumulado', 'Energia_Ativa']]

y_train4 = df_train4['Compensada']
y_train4 = np.array(y_train4).reshape(-1, 1)

X_test4 = df_test4[['Acumulado', 'Energia_Ativa']]

y_test4 = df_test4['Compensada']
y_test4 = np.array(y_test4).reshape(-1, 1)

### <b>2.4.5 XGBoost </b> <a class = "anchor" id = "chapter_2_4_5"></a>

In [ ]:
# Instanciando o modelo
modelo_xgb = xgb.XGBRegressor(random_state = 0, max_depth=2)

# Ajustando o modelo
modelo_xgb4 = modelo_xgb.fit(X_train4, y_train4)

# Fazendo as predicoes
predicao_xgb4 = modelo_xgb4.predict(X_test4)

print('XGBoost')
# r2 dos dados de treino
print('R² do treino = {}'.format(modelo_xgb4.score(X_train4, y_train4)))

# r2 dos dados de teste
print('R² do teste = %s' % metrics.r2_score(y_test4, predicao_xgb4))

# Erro medio quadratico dos dados de teste e preditos
print('Raiz quadrada do erro médio quadrático dos resultados preditos = {}'.format(np.sqrt(mean_squared_error(y_test4, predicao_xgb4))))

# Erro medio absoluto dos dados de teste e treino
print('Erro médio absoluto dos resultados preditos = {}'.format(mean_absolute_error(y_test4, predicao_xgb4).round(2)))

In [ ]:
# Verificando quais variaveis impactam mais o modelo

fi4 = pd.DataFrame(data=modelo_xgb4.feature_importances_,
             index=modelo_xgb4.feature_names_in_,
             columns=['importância'])
fi4.sort_values('importância').plot(kind='barh', title='Importância das Variáveis - 4')
plt.show()

In [ ]:
# Fazendo um gridsearch para encontrar os melhores parametros

# Intanciando o modelo
modelo_xgb = xgb.XGBRegressor(random_state = 0)

# Definindo o espaço de busca dos hiperparametros
param_grid = {'learning_rate': np.linspace(0.001, 0.1, num = 20), 'n_estimators': np.arange(100, 200, 10),
               'max_depth': [2, 3, 4, 5]}

# Chamando o griddsearch e fazendo a busca aleatoria de acordo com os hiperparametros fornecidos acima
grid_search4 =  GridSearchCV(modelo_xgb, param_grid, scoring='neg_mean_squared_error', cv = 5)
grid_search4.fit(X_train4, y_train4)

# Acessando os melhores hiperparametros encontrados
grid_search4.best_params_


In [ ]:
#Rodando o modelo novamente, agora com os novos parametros encontrados

# Instanciando o modelo
modelo_xgb = xgb.XGBRegressor(random_state = 0, n_estimators = 180, max_depth = 2, learning_rate = 0.027)

# Ajustando o modelo
modelo_xgb4 = modelo_xgb.fit(X_train4, y_train4)

# Fazendo as predicoes
predicao_xgb4 = modelo_xgb4.predict(X_test4)

print('XGBoost')
# r2 dos dados de treino
print('R² do treino = {}'.format(modelo_xgb4.score(X_train4, y_train4).round(2)))

# r2 dos dados de teste
print('R² do teste = %s' % metrics.r2_score(y_test4, predicao_xgb4).round(2))

# Erro quadratico medio dos dados de teste e preditos
print('Raiz quadrada do erro quadrático médio dos resultados preditos = {}'.format(np.sqrt(mean_squared_error(y_test4, predicao_xgb4)).round(2)))

# Erro medio absoluto dos dados de teste e treino
print('Erro médio absoluto dos resultados preditos = {}'.format(mean_absolute_error(y_test4, predicao_xgb4).round(2)))

In [ ]:
predictions_xgb4 = pd.DataFrame(y_test4, columns=['Compensada'])
predictions_xgb4.index = X_test4.index

predicao_xgb4 = pd.DataFrame(predicao_xgb4)
predicao_xgb4.index = X_test4.index

predictions_xgb4['Predição XGB'] = predicao_xgb4

In [ ]:
fig32 = go.Figure()

# Configurando o grafico de linhas
fig32.add_trace(go.Scatter(x = predictions_xgb4.index, y = predictions_xgb4['Compensada'], mode = 'lines + markers', name = 'Compensada', marker_color = '#266B6E'))
fig32.add_trace(go.Scatter(x = predictions_xgb4.index, y = predictions_xgb4['Predição XGB'], mode = 'lines + markers', name = 'Predição XGBoost', marker_color = '#FF007D'))

# Configurando o titulo do grafico
fig32.update_layout(title_text = f'Energia Compensada - 4',
                  title_font_size = 30,
                  title_x = 0.5)

# Configurando o titulo dos eixos
fig32.update_layout(xaxis_title_font_size = 18,
                  yaxis_title = 'Quantidade de Energia',
                  yaxis_title_font_size = 18,
                  showlegend = True,
                  legend_title = 'Legenda')

# Exibindo as grades de fundo
fig32.update_xaxes(showgrid = True)

# Plotando o grafico
fig32.show()

In [ ]:
erros_xgb4 = pd.DataFrame({'Métricas avaliadas': ['MAE', 'MAPE', 'WMAPE', 'MSE', 'RMSE'],
                          'Valores obtidos': [round(mean_absolute_error(y_test4, predicao_xgb4), 2),
                                              round(mean_absolute_percentage_error(y_test4, predicao_xgb4) * 100, 2),
                                              round(mean_absolute_percentage_error(y_test4, predicao_xgb4, sample_weight = y_test4) * 100, 2),
                                              round(mean_squared_error(y_test4, predicao_xgb4, squared = True), 2),
                                              round(mean_squared_error(y_test4, predicao_xgb4, squared = False), 2)]},
                        index = range(0, 5))

erros_xgb4

### <b>2.4.6 MLP </b> <a class = "anchor" id = "chapter_2_4_6"></a>

In [ ]:
# Instanciando o modelo
nnet = MLPRegressor(random_state = 0)

# Ajustando o modelo
modelo_nnet4 = nnet.fit(X_train4, y_train4)

# Fazendo as predicoes
predicao_nnet4 = modelo_nnet4.predict(X_test4)

print('MLP')
# r2 dos dados de treino
print('R² do treino = {}'.format(modelo_nnet4.score(X_train4, y_train4).round(2)))

# r2 dos dados de teste
print('R² do teste = %s' % metrics.r2_score(y_test4, predicao_nnet4).round(2))

In [ ]:
predictions_mlp4 = pd.DataFrame(y_test4, columns=['Compensada'])
predictions_mlp4.index = X_test4.index

predicao_nnet4 = pd.DataFrame(predicao_nnet4)
predicao_nnet4.index = X_test4.index

predictions_mlp4['Predição MLP'] = predicao_nnet4

In [ ]:
fig33 = go.Figure()

# Configurando o grafico de linhas
fig33.add_trace(go.Scatter(x = predictions_mlp4.index, y = predictions_mlp4['Compensada'], mode = 'lines + markers', name = 'Compensada', marker_color = '#266B6E'))
fig33.add_trace(go.Scatter(x = predictions_mlp4.index, y = predictions_mlp4['Predição MLP'], mode = 'lines + markers', name = 'Predição MLP', marker_color = '#FF007D'))

# Configurando o titulo do grafico
fig33.update_layout(title_text = f'Energia Compensada - 4',
                  title_font_size = 30,
                  title_x = 0.5)

# Configurando o titulo dos eixos
fig33.update_layout(xaxis_title_font_size = 18,
                  yaxis_title = 'Quantidade de Energia',
                  yaxis_title_font_size = 18,
                  showlegend = True,
                  legend_title = 'Legenda')

# Exibindo as grades de fundo
fig33.update_xaxes(showgrid = True)

# Plotando o grafico
fig33.show()

In [ ]:
erros_mlp4 = pd.DataFrame({'Métricas avaliadas': ['MAE', 'MAPE', 'WMAPE', 'MSE', 'RMSE'],
                          'Valores obtidos': [round(mean_absolute_error(y_test4, predicao_nnet4), 2),
                                              round(mean_absolute_percentage_error(y_test4, predicao_nnet4) * 100, 2),
                                              round(mean_absolute_percentage_error(y_test4, predicao_nnet4, sample_weight = y_test4) * 100, 2),
                                              round(mean_squared_error(y_test4, predicao_nnet4, squared = True), 2),
                                              round(mean_squared_error(y_test4, predicao_nnet4, squared = False), 2)]},
                        index = range(0, 5))

erros_mlp4

### <b>2.4.7 Random Forest </b> <a class = "anchor" id = "chapter_2_4_7"></a>

In [ ]:
# Instanciando o modelo
rf = RandomForestRegressor(random_state = 0)

# Ajustando o modelo
model_rf4 = rf.fit(X_train4, y_train4)

# Fazendo as predicoes
predicao_rf4 = model_rf4.predict(X_test4)

print('\n')

print('RANDOMFOREST')
# r2 dos dados de treino
print('R² do treino = {}'.format(model_rf4.score(X_train4, y_train4).round(2)))

# r2 dos dados de teste
print('R² do teste = %s' % metrics.r2_score(y_test4, predicao_rf4).round(2))

In [ ]:
predictions_rf4 = pd.DataFrame(y_test4, columns=['Compensada'])
predictions_rf4.index = X_test4.index

predicao_rf4 = pd.DataFrame(predicao_rf4)
predicao_rf4.index = X_test4.index

predictions_rf4['Predição Random Forest'] = predicao_rf4

In [ ]:
fig34 = go.Figure()

# Configurando o grafico de linhas
fig34.add_trace(go.Scatter(x = predictions_rf4.index, y = predictions_rf4['Compensada'], mode = 'lines + markers', name = 'Compensada', marker_color = '#266B6E'))
fig34.add_trace(go.Scatter(x = predictions_rf4.index, y = predictions_rf4['Predição Random Forest'], mode = 'lines + markers', name = 'Predição Random Forest', marker_color = '#FF007D'))

# Configurando o titulo do grafico
fig34.update_layout(title_text = f'Energia Compensada - 4',
                  title_font_size = 30,
                  title_x = 0.5)

# Configurando o titulo dos eixos
fig34.update_layout(xaxis_title_font_size = 18,
                  yaxis_title = 'Quantidade de Energia',
                  yaxis_title_font_size = 18,
                  showlegend = True,
                  legend_title = 'Legenda')

# Exibindo as grades de fundo
fig34.update_xaxes(showgrid = True)

# Plotando o grafico
fig34.show()

In [ ]:
erros_rf4 = pd.DataFrame({'Métricas avaliadas': ['MAE', 'MAPE', 'WMAPE', 'MSE', 'RMSE'],
                          'Valores obtidos': [round(mean_absolute_error(y_test4, predicao_rf4), 2),
                                              round(mean_absolute_percentage_error(y_test4, predicao_rf4) * 100, 2),
                                              round(mean_absolute_percentage_error(y_test4, predicao_rf4, sample_weight = y_test4) * 100, 2),
                                              round(mean_squared_error(y_test4, predicao_rf4, squared = True), 2),
                                              round(mean_squared_error(y_test4, predicao_rf4, squared = False), 2)]},
                        index = range(0, 5))

erros_rf4

### <b>2.4.8 Regressão Linear </b> <a class = "anchor" id = "chapter_2_4_8"></a>

In [ ]:
# Instanciando  o modelo 
rl = LinearRegression()

# Ajustando o modelo
modelo_rl4 = rl.fit(X_train4, y_train4)

# Fazendo as predicoes
predicao_rl4 = modelo_rl4.predict(X_test4)

print('REGRESSÃO LINEAR')
# r2 dos dados de treino
print('R² do treino = {}'.format(modelo_rl4.score(X_train4, y_train4).round(2)))

# r2 dos dados de teste
print('R² do teste = %s' % metrics.r2_score(y_test4, predicao_rl4).round(2))

In [ ]:
predictions_rl4 = pd.DataFrame(y_test4, columns=['Compensada'])
predictions_rl4.index = X_test4.index

predicao_rl4 = pd.DataFrame(predicao_rl4)
predicao_rl4.index = X_test4.index

predictions_rl4['Predição Regressão Linear'] = predicao_rl4

In [ ]:
fig35 = go.Figure()

# Configurando o grafico de linhas
fig35.add_trace(go.Scatter(x = predictions_rl4.index, y = predictions_rl4['Compensada'], mode = 'lines + markers', name = 'Compensada', marker_color = '#266B6E'))
fig35.add_trace(go.Scatter(x = predictions_rl4.index, y = predictions_rl4['Predição Regressão Linear'], mode = 'lines + markers', name = 'Predição Regressão Linear', marker_color = '#FF007D'))

# Configurando o titulo do grafico
fig35.update_layout(title_text = f'Energia Compensada - 4',
                  title_font_size = 30,
                  title_x = 0.5)

# Configurando o titulo dos eixos
fig35.update_layout(xaxis_title_font_size = 18,
                  yaxis_title = 'Quantidade de Energia',
                  yaxis_title_font_size = 18,
                  showlegend = True,
                  legend_title = 'Legenda')

# Exibindo as grades de fundo
fig35.update_xaxes(showgrid = True)

# Plotando o grafico
fig35.show()

In [ ]:
erros_rl4 = pd.DataFrame({'Métricas avaliadas': ['MAE', 'MAPE', 'WMAPE', 'MSE', 'RMSE'],
                          'Valores obtidos': [round(mean_absolute_error(y_test4, predicao_rl4), 2),
                                              round(mean_absolute_percentage_error(y_test4, predicao_rl4) * 100, 2),
                                              round(mean_absolute_percentage_error(y_test4, predicao_rl4, sample_weight = y_test4) * 100, 2),
                                              round(mean_squared_error(y_test4, predicao_rl4, squared = True), 2),
                                              round(mean_squared_error(y_test4, predicao_rl4, squared = False), 2)]},
                        index = range(0, 5))

erros_rl4

### <b>3. Resultados</b> <a class = "anchor" id = "chapter_3"></a>

# <b>3.1. Usina 1</b> <a class = "anchor" id = "chapter_3_1"></a>

In [ ]:
## Criando data frame com oas predicoes de todos os modelos

predicoes1 = pd.DataFrame(y_test1, columns=['Compensada'])
predicoes1.index = X_test1.index

predicao_xgb1 = pd.DataFrame(predicao_xgb1.round(2))
predicao_xgb1.index = X_test1.index
predicao_xgb_formatted1 = predicao_xgb1.applymap(lambda x: round(x, 2))

predicao_mlp1 = pd.DataFrame(predicao_nnet1.round(2))
predicao_mlp1.index = X_test1.index

predicao_rf1 = pd.DataFrame(predicao_rf1.round(2))
predicao_rf1.index = X_test1.index

predicao_rl1 = pd.DataFrame(predicao_rl1.round(2))
predicao_rl1.index = X_test1.index


predicao_holtwinters1 = pd.DataFrame(predictions_holtwinters1.round(2))
predicao_lstm1 = y_pred_df_lstm1['Compensada_predito'].apply(lambda x: round(x, 2))
predicao_sarimax1 = pd.DataFrame(predictions_sarimax1.round(2))
predicao_tensorflow1 = pd.DataFrame(predictions_tensorflow1.round(2))

predicoes1['Predição XGB'] = predicao_xgb_formatted1
predicoes1['Predição MLP'] = predicao_mlp1
predicoes1['Predição Random Forest'] = predicao_rf1
predicoes1['Predição Regressão Linear'] = predicao_rl1
predicoes1['Predição LSTM'] = predicao_lstm1
predicoes1['Predição Holt-Winters'] = predicao_holtwinters1
predicoes1['Predição SARIMAX'] = predicao_sarimax1
predicoes1['Predição TensorFlow'] = predicao_tensorflow1

predicoes1

In [ ]:
# Criando dataframe com os valores de erros de todos os modelos

erros1 = pd.DataFrame({
    'LSTM': erros_lstm1['Valores obtidos'],
    'Holt-Winters': erros_hw1['Valores obtidos'],
    'SARIMAX': erros_sm1['Valores obtidos'],
    'TensorFlow': erros_tf1['Valores obtidos'],
    'XGBoost': erros_xgb1['Valores obtidos'],
    'MLP': erros_mlp1['Valores obtidos'],
    'Random Forest': erros_rf1['Valores obtidos'],
    'Regressão Linear': erros_rl1['Valores obtidos'],
})

# Definindo a coluna 'Metricas avaliadas' como índice
erros1.set_index(erros_tf1['Métricas avaliadas'], inplace=True)

erros1

In [ ]:
fig36 = go.Figure()

# Configurando o grafico de linhas
fig36.add_trace(go.Scatter(x = predicoes1.index, y = predicoes1['Compensada'], mode = 'lines + markers', name = 'Compensada', marker_color = '#266B6E'))
fig36.add_trace(go.Scatter(x = predicoes1.index, y = predicoes1['Predição SARIMAX'], mode = 'lines + markers', name = 'Predição Sarimax', marker_color = '#FF007D'))
fig36.add_trace(go.Scatter(x = predicoes1.index, y = predicoes1['Predição Holt-Winters'], mode = 'lines + markers', name = 'Predição Holt-Winters'))
fig36.add_trace(go.Scatter(x = predicoes1.index, y = predicoes1['Predição LSTM'], mode = 'lines + markers', name = 'Predição LSTM'))
fig36.add_trace(go.Scatter(x = predicoes1.index, y = predicoes1['Predição TensorFlow'], mode = 'lines + markers', name = 'Predição Tensorflow'))

fig36.add_trace(go.Scatter(x = predicoes1.index, y = predicoes1['Predição XGB'], mode = 'lines + markers', name = 'Predição XGBoost'))
fig36.add_trace(go.Scatter(x = predicoes1.index, y = predicoes1['Predição MLP'], mode = 'lines + markers', name = 'Predição MLP'))
fig36.add_trace(go.Scatter(x = predicoes1.index, y = predicoes1['Predição Random Forest'], mode = 'lines + markers', name = 'Predição Random Forest'))
fig36.add_trace(go.Scatter(x = predicoes1.index, y = predicoes1['Predição Regressão Linear'], mode = 'lines + markers', name = 'Predição Regressão Linear'))


# Configurando o titulo do grafico
fig36.update_layout(title_text = f'Energia Compensada - 1',
                  title_font_size = 30,
                  title_x = 0.5)

# Configurando o titulo dos eixos
fig36.update_layout(xaxis_title_font_size = 18,
                  yaxis_title = 'Quantidade de Energia',
                  yaxis_title_font_size = 18,
                  showlegend = True,
                  legend_title = 'Legenda')

# Exibindo as grades de fundo
fig36.update_xaxes(showgrid = True)

# Plotando o grafico
fig36.show()

### <b>3.2. Usina  2 </b> <a class = "anchor" id = "chapter_3_2"></a>

In [ ]:
## Criando data frame com oas predicoes de todos os modelos

predicoes2 = pd.DataFrame(y_test2, columns=['Compensada'])
predicoes2.index = X_test2.index

predicao_xgb2 = pd.DataFrame(predicao_xgb2.round(2))
predicao_xgb2.index = X_test2.index
predicao_xgb_formatted2 = predicao_xgb2.applymap(lambda x: round(x, 2))

predicao_mlp2 = pd.DataFrame(predicao_nnet2.round(2))
predicao_mlp2.index = X_test2.index

predicao_rf2 = pd.DataFrame(predicao_rf2.round(2))
predicao_rf2.index = X_test2.index

predicao_rl2 = pd.DataFrame(predicao_rl2.round(2))
predicao_rl2.index = X_test2.index


predicao_holtwinters2 = pd.DataFrame(predictions_holtwinters2.round(2))
predicao_lstm2 = y_pred_df_lstm2['Compensada_predito'].apply(lambda x: round(x, 2))
predicao_sarimax2 = pd.DataFrame(predictions_sarimax2.round(2))
predicao_tensorflow2 = pd.DataFrame(predictions_tensorflow2.round(2))

predicoes2['Predição XGB'] = predicao_xgb_formatted2
predicoes2['Predição MLP'] = predicao_mlp2
predicoes2['Predição Random Forest'] = predicao_rf2
predicoes2['Predição Regressão Linear'] = predicao_rl2
predicoes2['Predição LSTM'] = predicao_lstm2
predicoes2['Predição Holt-Winters'] = predicao_holtwinters2
predicoes2['Predição SARIMAX'] = predicao_sarimax2
predicoes2['Predição TensorFlow'] = predicao_tensorflow2

predicoes2

In [ ]:
# Criando dataframe com os valores de erros de todos os modelos

erros2 = pd.DataFrame({
    'LSTM': erros_lstm2['Valores obtidos'],
    'Holt-Winters': erros_hw2['Valores obtidos'],
    'SARIMAX': erros_sm2['Valores obtidos'],
    'TensorFlow': erros_tf2['Valores obtidos'],
    'XGBoost': erros_xgb2['Valores obtidos'],
    'MLP': erros_mlp2['Valores obtidos'],
    'Random Forest': erros_rf2['Valores obtidos'],
    'Regressão Linear': erros_rl2['Valores obtidos'],
})

# Definindo a coluna 'Metricas avaliadas' como índice
erros2.set_index(erros_tf2['Métricas avaliadas'], inplace=True)

erros2

In [ ]:
fig37 = go.Figure()

# Configurando o grafico de linhas
fig37.add_trace(go.Scatter(x = predicoes2.index, y = predicoes2['Compensada'], mode = 'lines + markers', name = 'Compensada', marker_color = '#266B6E'))
fig37.add_trace(go.Scatter(x = predicoes2.index, y = predicoes2['Predição SARIMAX'], mode = 'lines + markers', name = 'Predição Sarimax', marker_color = '#FF007D'))
fig37.add_trace(go.Scatter(x = predicoes2.index, y = predicoes2['Predição Holt-Winters'], mode = 'lines + markers', name = 'Predição Holt-Winters'))
fig37.add_trace(go.Scatter(x = predicoes2.index, y = predicoes2['Predição LSTM'], mode = 'lines + markers', name = 'Predição LSTM'))
fig37.add_trace(go.Scatter(x = predicoes2.index, y = predicoes2['Predição TensorFlow'], mode = 'lines + markers', name = 'Predição Tensorflow'))

fig37.add_trace(go.Scatter(x = predicoes2.index, y = predicoes2['Predição XGB'], mode = 'lines + markers', name = 'Predição XGBoost'))
fig37.add_trace(go.Scatter(x = predicoes2.index, y = predicoes2['Predição MLP'], mode = 'lines + markers', name = 'Predição MLP'))
fig37.add_trace(go.Scatter(x = predicoes2.index, y = predicoes2['Predição Random Forest'], mode = 'lines + markers', name = 'Predição Random Forest'))
fig37.add_trace(go.Scatter(x = predicoes2.index, y = predicoes2['Predição Regressão Linear'], mode = 'lines + markers', name = 'Predição Regressão Linear'))


# Configurando o titulo do grafico
fig37.update_layout(title_text = f'Energia Compensada -  2',
                  title_font_size = 30,
                  title_x = 0.5)

# Configurando o titulo dos eixos
fig37.update_layout(xaxis_title_font_size = 18,
                  yaxis_title = 'Quantidade de Energia',
                  yaxis_title_font_size = 18,
                  showlegend = True,
                  legend_title = 'Legenda')

# Exibindo as grades de fundo
fig37.update_xaxes(showgrid = True)

# Plotando o gráfico
fig37.show()

### <b>3.3. Usina 3 </b> <a class = "anchor" id = "chapter_3_3"></a>

In [ ]:
## Criando data frame com oas predicoes de todos os modelos

predicoes3 = pd.DataFrame(y_test3, columns=['Compensada'])
predicoes3.index = X_test3.index

predicao_xgb3 = pd.DataFrame(predicao_xgb3.round(2))
predicao_xgb3.index = X_test3.index
predicao_xgb_formatted3 = predicao_xgb3.applymap(lambda x: round(x, 2))

predicao_mlp3 = pd.DataFrame(predicao_nnet3.round(2))
predicao_mlp3.index = X_test3.index

predicao_rf3 = pd.DataFrame(predicao_rf3.round(2))
predicao_rf3.index = X_test3.index

predicao_rl3 = pd.DataFrame(predicao_rl3.round(2))
predicao_rl3.index = X_test3.index


predicao_holtwinters3 = pd.DataFrame(predictions_holtwinters3.round(2))
predicao_lstm3 = y_pred_df_lstm3['Compensada_predito'].apply(lambda x: round(x, 2))
predicao_sarimax3 = pd.DataFrame(predictions_sarimax3.round(2))
predicao_tensorflow3 = pd.DataFrame(predictions_tensorflow3.round(2))

predicoes3['Predição XGB'] = predicao_xgb_formatted3
predicoes3['Predição MLP'] = predicao_mlp3
predicoes3['Predição Random Forest'] = predicao_rf3
predicoes3['Predição Regressão Linear'] = predicao_rl3
predicoes3['Predição LSTM'] = predicao_lstm3
predicoes3['Predição Holt-Winters'] = predicao_holtwinters3
predicoes3['Predição SARIMAX'] = predicao_sarimax3
predicoes3['Predição TensorFlow'] = predicao_tensorflow3

predicoes3

In [ ]:
# Criando dataframe com os valores de erros de todos os modelos

erros3 = pd.DataFrame({
    'LSTM': erros_lstm3['Valores obtidos'],
    'Holt-Winters': erros_hw3['Valores obtidos'],
    'SARIMAX': erros_sm3['Valores obtidos'],
    'TensorFlow': erros_tf3['Valores obtidos'],
    'XGBoost': erros_xgb3['Valores obtidos'],
    'MLP': erros_mlp3['Valores obtidos'],
    'Random Forest': erros_rf3['Valores obtidos'],
    'Regressão Linear': erros_rl3['Valores obtidos'],
})

# Definindo a coluna 'Metricas avaliadas' como índice
erros3.set_index(erros_tf3['Métricas avaliadas'], inplace=True)

erros3

In [ ]:
fig38 = go.Figure()

# Configurando o grafico de linhas
fig38.add_trace(go.Scatter(x = predicoes3.index, y = predicoes3['Compensada'], mode = 'lines + markers', name = 'Compensada', marker_color = '#266B6E'))
fig38.add_trace(go.Scatter(x = predicoes3.index, y = predicoes3['Predição SARIMAX'], mode = 'lines + markers', name = 'Predição Sarimax', marker_color = '#FF007D'))
fig38.add_trace(go.Scatter(x = predicoes3.index, y = predicoes3['Predição Holt-Winters'], mode = 'lines + markers', name = 'Predição Holt-Winters'))
fig38.add_trace(go.Scatter(x = predicoes3.index, y = predicoes3['Predição LSTM'], mode = 'lines + markers', name = 'Predição LSTM'))
fig38.add_trace(go.Scatter(x = predicoes3.index, y = predicoes3['Predição TensorFlow'], mode = 'lines + markers', name = 'Predição Tensorflow'))

fig38.add_trace(go.Scatter(x = predicoes3.index, y = predicoes3['Predição XGB'], mode = 'lines + markers', name = 'Predição XGBoost'))
fig38.add_trace(go.Scatter(x = predicoes3.index, y = predicoes3['Predição MLP'], mode = 'lines + markers', name = 'Predição MLP'))
fig38.add_trace(go.Scatter(x = predicoes3.index, y = predicoes3['Predição Random Forest'], mode = 'lines + markers', name = 'Predição Random Forest'))
fig38.add_trace(go.Scatter(x = predicoes3.index, y = predicoes3['Predição Regressão Linear'], mode = 'lines + markers', name = 'Predição Regressão Linear'))


# Configurando o titulo do grafico
fig38.update_layout(title_text = f'Energia Compensada - 3',
                  title_font_size = 30,
                  title_x = 0.5)

# Configurando o titulo dos eixos
fig38.update_layout(xaxis_title_font_size = 18,
                  yaxis_title = 'Quantidade de Energia',
                  yaxis_title_font_size = 18,
                  showlegend = True,
                  legend_title = 'Legenda')

# Exibindo as grades de fundo
fig38.update_xaxes(showgrid = True)

# Plotando o grafico
fig38.show()

### <b>3.4. Usina 4 </b> <a class = "anchor" id = "chapter_3_4"></a>

In [ ]:
## Criando data frame com oas predicoes de todos os modelos

predicoes4 = pd.DataFrame(y_test4, columns=['Compensada'])
predicoes4.index = X_test4.index

predicao_xgb4 = pd.DataFrame(predicao_xgb4.round(2))
predicao_xgb4.index = X_test4.index
predicao_xgb_formatted4 = predicao_xgb4.applymap(lambda x: round(x, 2))

predicao_mlp4 = pd.DataFrame(predicao_nnet4.round(2))
predicao_mlp4.index = X_test4.index

predicao_rf4 = pd.DataFrame(predicao_rf4.round(2))
predicao_rf4.index = X_test4.index

predicao_rl4 = pd.DataFrame(predicao_rl4.round(2))
predicao_rl4.index = X_test4.index


predicao_holtwinters4 = pd.DataFrame(predictions_holtwinters4.round(2))
predicao_lstm4 = y_pred_df_lstm4['Compensada_predito'].apply(lambda x: round(x, 2))
predicao_sarimax4 = pd.DataFrame(predictions_sarimax4.round(2))
predicao_tensorflow4 = pd.DataFrame(predictions_tensorflow4.round(2))

predicoes4['Predição XGB'] = predicao_xgb_formatted4
predicoes4['Predição MLP'] = predicao_mlp4
predicoes4['Predição Random Forest'] = predicao_rf4
predicoes4['Predição Regressão Linear'] = predicao_rl4
predicoes4['Predição LSTM'] = predicao_lstm4
predicoes4['Predição Holt-Winters'] = predicao_holtwinters4
predicoes4['Predição SARIMAX'] = predicao_sarimax4
predicoes4['Predição TensorFlow'] = predicao_tensorflow4

predicoes4

In [ ]:
# Criando dataframe com os valores de erros de todos os modelos

erros4 = pd.DataFrame({
    'LSTM': erros_lstm4['Valores obtidos'],
    'Holt-Winters': erros_hw4['Valores obtidos'],
    'SARIMAX': erros_sm4['Valores obtidos'],
    'TensorFlow': erros_tf4['Valores obtidos'],
    'XGBoost': erros_xgb4['Valores obtidos'],
    'MLP': erros_mlp4['Valores obtidos'],
    'Random Forest': erros_rf4['Valores obtidos'],
    'Regressão Linear': erros_rl4['Valores obtidos'],
})

# Definindo a coluna 'Metricas avaliadas' como índice
erros4.set_index(erros_tf4['Métricas avaliadas'], inplace=True)

erros4

In [ ]:
fig39 = go.Figure()

# Configurando o grafico de linhas
fig39.add_trace(go.Scatter(x = predicoes4.index, y = predicoes4['Compensada'], mode = 'lines + markers', name = 'Compensada', marker_color = '#266B6E'))
fig39.add_trace(go.Scatter(x = predicoes4.index, y = predicoes4['Predição SARIMAX'], mode = 'lines + markers', name = 'Predição Sarimax', marker_color = '#FF007D'))
fig39.add_trace(go.Scatter(x = predicoes4.index, y = predicoes4['Predição Holt-Winters'], mode = 'lines + markers', name = 'Predição Holt-Winters'))
fig39.add_trace(go.Scatter(x = predicoes4.index, y = predicoes4['Predição LSTM'], mode = 'lines + markers', name = 'Predição LSTM'))
fig39.add_trace(go.Scatter(x = predicoes4.index, y = predicoes4['Predição TensorFlow'], mode = 'lines + markers', name = 'Predição Tensorflow'))

fig39.add_trace(go.Scatter(x = predicoes4.index, y = predicoes4['Predição XGB'], mode = 'lines + markers', name = 'Predição XGBoost'))
fig39.add_trace(go.Scatter(x = predicoes4.index, y = predicoes4['Predição MLP'], mode = 'lines + markers', name = 'Predição MLP'))
fig39.add_trace(go.Scatter(x = predicoes4.index, y = predicoes4['Predição Random Forest'], mode = 'lines + markers', name = 'Predição Random Forest'))
fig39.add_trace(go.Scatter(x = predicoes4.index, y = predicoes4['Predição Regressão Linear'], mode = 'lines + markers', name = 'Predição Regressão Linear'))


# Configurando o titulo do grafico
fig39.update_layout(title_text = f'Energia Compensada - 4',
                  title_font_size = 30,
                  title_x = 0.5)

# Configurando o titulo dos eixos
fig39.update_layout(xaxis_title_font_size = 18,
                  yaxis_title = 'Quantidade de Energia',
                  yaxis_title_font_size = 18,
                  showlegend = True,
                  legend_title = 'Legenda')

# Exibindo as grades de fundo
fig39.update_xaxes(showgrid = True)

# Plotando o grafico
fig39.show()